# NEW IMPLEMENTATION

# OLD IMPLEMENTATION

In [8]:
# !pip install folium routingpy openrouteservice geopy 

In [ ]:
import pandas as pd 
from src.get_data import get_data 
from src.get_data import get_connection
from datetime import datetime, timedelta
from src.routing.routing_optimizer import RouteOptimizer
from src.routing.routing import get_valhalla_routes_info, plot_routes_on_map
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score 
import folium
from src.map_viz.plot_cluster import create_enhanced_cluster_map
import openrouteservice as ors
import math
import numpy as np
import os 
from routingpy import Valhalla
# client = ors.Client(key='5b3ce3597851110001cf62485a415b103df64104ad2680c9210ef936') 

import logging
import pandas as pd
from pyodbc import Connection

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# from src.route_optimization import run_route_optimizer


VALHALLA_BASE_URL = "http://localhost:8002" # Pointing to your self-hosted Valhalla
VALHALLA_API_KEY = "" # No API key needed for your self-hosted instance


CURRENT_DATE  = datetime.today().date() # + timedelta(days=1)

## **Utils**

#### Preprocessing Functions

In [21]:
def clean_invalid_coordinates_DEP(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces invalid Latitude (< -90 or > 90) and Longitude (< -180 or > 180) values with 0.0.

    Args:
        df (pd.DataFrame): Input DataFrame with 'Latitude' and 'Longitude' columns.

    Returns:
        pd.DataFrame: DataFrame with corrected coordinate values.
    """
    df = df.copy()

    df.loc[(df['Latitude'] < -90) | (df['Latitude'] > 90), 'Latitude'] = 0.0
    df.loc[(df['Longitude'] < -180) | (df['Longitude'] > 180), 'Longitude'] = 0.0


    ### Nigeria 
    # ADD NIGERIA FILTER HERE
    return df

def clean_invalid_coordinates(df: pd.DataFrame, offset_degrees: float = 0.1) -> pd.DataFrame:
    """
    Replaces invalid Latitude (< -90 or > 90) and Longitude (< -180 or > 180) values with 0.0.
    Also replaces coordinates outside Nigeria's approximate boundaries (with an optional offset) with 0.0.

    Args:
        df (pd.DataFrame): Input DataFrame with 'Latitude' and 'Longitude' columns.
        offset_degrees (float): Degrees to add/subtract from the strict Nigeria boundary
                                to expand the bounding box. Default is 0.1 degrees.

    Returns:
        pd.DataFrame: DataFrame with corrected coordinate values.
    """
    df = df.copy()

    # Global invalid coordinate ranges
    df.loc[(df['Latitude'] < -90) | (df['Latitude'] > 90), 'Latitude'] = 0.0
    df.loc[(df['Longitude'] < -180) | (df['Longitude'] > 180), 'Longitude'] = 0.0

    ### Nigeria Boundary Filter ###
    # Approximate decimal degree boundaries for Nigeria
    STRICT_NIGERIA_MIN_LAT = 4.10
    STRICT_NIGERIA_MAX_LAT = 13.90
    STRICT_NIGERIA_MIN_LON = 2.60
    STRICT_NIGERIA_MAX_LON = 14.70

    # Apply offset to expand the bounding box
    NIGERIA_MIN_LAT = STRICT_NIGERIA_MIN_LAT - offset_degrees
    NIGERIA_MAX_LAT = STRICT_NIGERIA_MAX_LAT + offset_degrees
    NIGERIA_MIN_LON = STRICT_NIGERIA_MIN_LON - offset_degrees
    NIGERIA_MAX_LON = STRICT_NIGERIA_MAX_LON + offset_degrees

    # Identify coordinates outside Nigeria's expanded bounding box
    # Condition for rows outside Nigeria's latitude range
    outside_nigeria_lat = (df['Latitude'] < NIGERIA_MIN_LAT) | \
                          (df['Latitude'] > NIGERIA_MAX_LAT)

    # Condition for rows outside Nigeria's longitude range
    outside_nigeria_lon = (df['Longitude'] < NIGERIA_MIN_LON) | \
                          (df['Longitude'] > NIGERIA_MAX_LON)

    # Combine conditions: if EITHER latitude OR longitude is outside Nigeria's expanded box,
    # then set BOTH Latitude and Longitude for that row to 0.0.
    # We apply this only to coordinates that are already globally valid (i.e., not 0.0).
    df.loc[
        (df['Latitude'] != 0.0) &
        (df['Longitude'] != 0.0) &
        (outside_nigeria_lat | outside_nigeria_lon),
        ['Latitude', 'Longitude']
    ] = 0.0

    return df

In [22]:
def preprocessing(df_customer_sku_recommendation_raw, 
                      df_customer_dim_with_affinity_score_raw, 
                      df_stockpoint_dim_raw,
                      df_customer_score,
                      df_kyc_customer) :
    
    df_customer_sku_recommendation_raw['Stock_Point_ID'] = df_customer_sku_recommendation_raw['Stock_Point_ID'].astype(int)
    df_customer_dim_with_affinity_score_raw['Stock_Point_ID'] = df_customer_dim_with_affinity_score_raw['Stock_Point_ID'].astype(int)
    df_stockpoint_dim_raw['Stock_Point_ID'] = df_stockpoint_dim_raw['Stock_Point_ID'].astype(int)
    df_customer_score = df_customer_score.rename(columns={'StockPointID':'Stock_Point_ID'})
    df_customer_score['Stock_Point_ID'] = df_customer_score['Stock_Point_ID'].astype(int)


    # ----------------- CUSTOMER DIM TABLE 
    col_sel_affinity = ['Region', 'Stock_Point_ID', 'CustomerID']

    col_sel_kyc = ['CustomerID', 'ContactName', 'BusinessName', 'CustomerModeName',
        'CustomerRef', 'ContactPhone', 'CustomerType', 'FullAddress', 
        'StateName', 'CityName', 'TownName', 'Latitude','Longitude', 
        'DistanceVarianceInMeter', 'IsLocationSubmitted',
        'IsLocationCaptured', 'IsLocationVerified','CustomerStatus',
        'RejectReason',  'KYC_Capture_Status',  'lastDelvDate', 
        # 'hasPOS','hasVAS', 'hasBNPL', 'lastDelvDate', 
        'isActive']

    col_sel_score = ['Stock_Point_ID', 'CustomerID', 'composite_customer_score',
        'percentile_rank', 'active_months_pct', 'avg_orders_per_active_month',
        'avg_qty_per_month', 'avg_revenue_per_month', 'days_since_last_order']

    df_master_customer_dim = (
                df_customer_dim_with_affinity_score_raw[col_sel_affinity]
                .merge(df_kyc_customer[col_sel_kyc], how='inner', on=['CustomerID'])
                .merge(df_customer_score[col_sel_score], how='left', on=['Stock_Point_ID', 'CustomerID'])
                .rename(columns = {'CityName':'LGA',
                                'TownName':'LCDA'
                                })

            )

    # Change CustomerPurchaseRecency from lastDelvDate to days since last order (order creation date)
    df_master_customer_dim['CustomerPurchaseRecency'] =  df_master_customer_dim['days_since_last_order']
    # df_master_customer_dim['CustomerPurchaseRecency'] =  df_master_customer_dim['lastDelvDate'].apply(lambda x: (datetime.now() - x).days)
    df_master_customer_dim['CustomerPurchaseRecency'] = df_master_customer_dim['CustomerPurchaseRecency'].fillna(max(df_master_customer_dim['CustomerPurchaseRecency']))
    df_master_customer_dim['KYC_Capture_Status'] = df_master_customer_dim['KYC_Capture_Status'].apply(lambda x: 'Yes' if x == 1 else 'No')

    # Add to Score
    # Fix Missing value -------------------------------------------
    for col in ['BusinessName', 'CustomerModeName', 'FullAddress', 'LGA', 'LCDA']:
        df_master_customer_dim[col] = df_master_customer_dim[col].fillna('')

    for col in ['Latitude',  'Longitude', 'composite_customer_score',  
                'percentile_rank',  'active_months_pct', 'avg_orders_per_active_month',  
                'avg_qty_per_month',  'avg_revenue_per_month'
                ]:
        df_master_customer_dim[col] = pd.to_numeric(df_master_customer_dim[col], errors='coerce').fillna(0) 

    df_master_customer_dim = clean_invalid_coordinates(df_master_customer_dim)
    
    # Add to Score 
    # Boost composite score and percentile rank for customers with completed KYC
    mask_kyc = df_master_customer_dim['KYC_Capture_Status'] == 'Yes'

    df_master_customer_dim.loc[mask_kyc, 'composite_customer_score'] += 5
    df_master_customer_dim.loc[mask_kyc, 'percentile_rank'] += 0.1 

    # ----------------- RECOMMENDATION
    col2 = ['EstimatedQuantity', 'CustomerSKUscore', 'CustomerSKUscoreStandardize', 'CustomerSKUscoreRank']
    for col in col2: 
        df_customer_sku_recommendation_raw[col] = pd.to_numeric(df_customer_sku_recommendation_raw[col], errors='coerce')

    df_customer_sku_recommendation_raw['LastDeliveredDate'] = pd.to_datetime(df_customer_sku_recommendation_raw['LastDeliveredDate'])
    # Get today's date
    today = pd.Timestamp.today()

    df_customer_sku_recommendation_raw['Recency'] = df_customer_sku_recommendation_raw['LastDeliveredDate'].apply(lambda x: (datetime.now() - x).days)
    df_customer_sku_recommendation_raw['Recency'] = df_customer_sku_recommendation_raw['Recency'].fillna(max(df_customer_sku_recommendation_raw['Recency']))
    
    # ----------------- STOCKPOINT
    df_stockpoint_dim_raw.rename(columns={'lattitude':'Latitude', 'longitude':'Longitude'}, inplace=True) 
    col3 = ['Latitude', 'Longitude']
    for col in col3: 
        df_stockpoint_dim_raw[col] = pd.to_numeric(df_stockpoint_dim_raw[col], errors='coerce').fillna(0)    

    # Replace invalid latitude values with NaN
    df_stockpoint_dim_raw = clean_invalid_coordinates(df_stockpoint_dim_raw)   
    

    return df_customer_sku_recommendation_raw, df_master_customer_dim, df_stockpoint_dim_raw

In [23]:
def data_filter(df_customer_sku_recommendation, df_master_customer_dim, df_stockpoint_dim,
                stockpoint_id,  sku_recency = 7, customer_recency = 90, number_recommendation = 5,
                estimate_qty_scale_factor = .90, max_estimated_qty = 5, exclude_recency_customer = 4):
    
    df_customer_sku_recommendation = df_customer_sku_recommendation.copy().query(f'Stock_Point_ID == {stockpoint_id}')
    # Filter Recommendation
    df_customer_sku_recommendation = df_customer_sku_recommendation[df_customer_sku_recommendation['ProductTag'] != 'Standard-Inactive']
    df_customer_sku_recommendation = df_customer_sku_recommendation[df_customer_sku_recommendation['Medium'] != 'Never Purchased']

    # Filter customer base
    df_master_customer_dim['valid_for_push'] = np.where(
                                                    #  df_master_customer_dim['KYC_Capture_Status'] == 'Yes'   
                                                    (
                                                        (df_master_customer_dim['IsLocationCaptured'] == 'Yes') |
                                                        (df_master_customer_dim['DistanceVarianceInMeter'] <= 150.0) |
                                                        (df_master_customer_dim['KYC_Capture_Status'] == 'Yes') |
                                                        (df_master_customer_dim['CustomerPurchaseRecency'] <= customer_recency)
                                                    )
                                                    ,1,0
                                                )
    # df_master_customer_dim = df_master_customer_dim[df_master_customer_dim['CustomerPurchaseRecency'] <= customer_recency]
    df_master_customer_dim = df_master_customer_dim.query('valid_for_push == 1')  
    # Exclude Customer with recent purchase of any SKU
    df_master_customer_dim = df_master_customer_dim.query(f'CustomerPurchaseRecency > {exclude_recency_customer}')
    # Customer with valid Location Coordination
    df_master_customer_dim = df_master_customer_dim.query('Latitude != 0').reset_index(drop=True)
    
    # # Clipping Max Estimated Quantity to 10 qty
    df_customer_sku_recommendation['EstimatedQuantity_bck'] = df_customer_sku_recommendation['EstimatedQuantity']
    df_customer_sku_recommendation['EstimatedQuantity'] = df_customer_sku_recommendation['EstimatedQuantity'].apply(lambda x: max_estimated_qty if int((x*estimate_qty_scale_factor)) > max_estimated_qty else int((x*estimate_qty_scale_factor)) )


    # Select top 10 SKU by SKURank per customer
    df_customer_sku_recommendation = (
        df_customer_sku_recommendation
        .query('EstimatedQuantity > 1')
        .sort_values(['CustomerID','CustomerSKUscoreRank'])
        .groupby('CustomerID', group_keys=False)
        .head(number_recommendation)
        .reset_index(drop=True) 
    )

    df_customer_sku_recommendation_ = df_master_customer_dim.merge(df_customer_sku_recommendation, how='inner', on = ['CustomerID','Stock_Point_ID'])  

    df_stockpoint_dim = df_stockpoint_dim.query(f'Stock_Point_ID == {stockpoint_id}').reset_index(drop=True) 
    

    df_customer_dim = df_master_customer_dim.merge(df_customer_sku_recommendation_['CustomerID'].drop_duplicates(), how='inner', on = 'CustomerID')
    # df_customer_dim = df_customer_dim.merge(df_customer_dim_with_affinity_score[sel_cols], how='inner', on = 'CustomerID').reset_index(drop = True) 
    
    print(f'Total Quantity before filter: {df_customer_sku_recommendation.query(f"Stock_Point_ID == {stockpoint_id}").EstimatedQuantity.sum():,}')
    print(f'Total Quantity: {df_customer_sku_recommendation_.EstimatedQuantity.sum():,}')
    print(f'Total Number of Customers before filter: {df_customer_sku_recommendation.query(f"Stock_Point_ID == {stockpoint_id}").CustomerID.nunique():,}')
    print(f'Total Number of Customers: {df_customer_dim.CustomerID.nunique():,}')

 
    return df_customer_sku_recommendation_, df_customer_dim,   df_stockpoint_dim  

In [24]:
def export_data(
        selected_trip,
        all_push_recommendation,
        cluster_summary,
        stock_point_name
    ): 
    dir_path = f'./recommendation_output/{CURRENT_DATE}'
    
    # Ensure directory exists
    os.makedirs(dir_path, exist_ok=True)
    
    file_path = f'{dir_path}/{stock_point_name}_{CURRENT_DATE}.xlsx'

    with pd.ExcelWriter(file_path) as writer:
        selected_trip.to_excel(writer, sheet_name='Selected Trip', index=False)
        all_push_recommendation.to_excel(writer, sheet_name='All Recommendation', index=False)
        cluster_summary.to_excel(writer, sheet_name='Recommendation Cluster Summary', index=False)


#### Map-Utils

In [ ]:
def vis_and_save(df_routes, 
                 df_stockpoint = None,   
                 filename=None,
                 cluster_col='cluster',
                 popup_cols=['CustomerID', 'LGA', 'LCDA'],
                 tooltip_cols=['LGA', 'LCDA'],
                 depot_marker_size=10
                 ):
    
    map_clusters = create_enhanced_cluster_map(
        df_routes,
        popup_cols=popup_cols,
        tooltip_cols=tooltip_cols, 
        cluster_col = cluster_col,
        zoom_start=10, 
        radius=8
    )
    
    if df_stockpoint:
        depot_location = [df_stockpoint.Latitude[0], df_stockpoint.Longitude[0]]
        depot_name = df_stockpoint.Stock_point_Name[0]
        map_clusters = map_clusters.add_child(folium.Marker(location=depot_location, 
                                size = depot_marker_size, 
                                tooltip=depot_name, 
                                icon=folium.Icon(color="green", 
                                icon="home")))  
    if filename:
        map_clusters.save(filename)
    return map_clusters

#### Cluster Summary Route-Function

In [26]:
def evaluate_unsupervised_clustering(df):
    # Usage:
    X = df[['Latitude', 'Longitude']].values
    labels = df['cluster'].values
    scores = {
        "Silhouette Score":  silhouette_score(X, labels).round(2),
        "Davies-Bouldin Index": davies_bouldin_score(X, labels).round(2),
        "Calinski-Harabasz Score": calinski_harabasz_score(X, labels).round(2)
    }

    for key in scores:
        print(f"{key}: {scores[key]}")
    return scores

In [27]:
def create_stockpoint_dict(df_selected_trip, df_stockpoint_dim):
    """
    Create a dictionary structure with stock point information and associated trips.
    
    Parameters:
    df_selected_trip: DataFrame with columns ['StockPointID', 'StockPointName', 'TripID', 'CustomerID', 'Latitude', 'Longitude', 'EstimatedQuantity']
    df_stockpoint_dim: DataFrame with columns ['Stock_Point_ID', 'Stock_point_Name', 'Latitude', 'Longitude']
    
    Returns:
    dict: Dictionary with stock point information and trips
    """
    if df_selected_trip.empty:
        logger.info('Dataframe is empty')
        return {}
    
    # Group by StockPointID to handle each stock point
    stockpoint_groups = df_selected_trip.groupby('StockPointID')
    
    result = {}
    
    for stock_point_id, group in stockpoint_groups:
        # Get stock point information from df_stockpoint_dim
        stock_point_info = df_stockpoint_dim[df_stockpoint_dim['Stock_Point_ID'] == stock_point_id]
        
        if stock_point_info.empty:
            # If stock point not found in dimension table, use info from selected_trip
            stock_point_name = group['StockPointName'].iloc[0]
            # Note: We'll need to get coordinates from somewhere since customer coordinates 
            # in df_selected_trip are for destinations, not stock points
            stock_point_coord = [0, 0]  # Placeholder - you may need to adjust this
        else:
            stock_point_name = stock_point_info['Stock_point_Name'].iloc[0]
            stock_point_coord = [
                stock_point_info['Longitude'].iloc[0], 
                stock_point_info['Latitude'].iloc[0]
            ]
        
        # Group by TripID to organize trips
        trip_groups = group.groupby('TripID')
        trips = []
        
        for trip_id, trip_group in trip_groups:
            # Create destinations list for this trip
            destinations = []
            for _, row in trip_group.iterrows():
                destination = {
                    'CustomerID': row['CustomerID'],
                    'Coordinate': [row['Longitude'], row['Latitude']]
                }
                destinations.append(destination)
            
            # Create trip dictionary
            trip_dict = {
                'TripID': trip_id,
                'Destinations': destinations
            }
            trips.append(trip_dict)
        
        # Create the final dictionary structure for this stock point
        result[stock_point_id] = {
            'StockPointName': stock_point_name,
            'StockPointID': stock_point_id,
            'StockPointCoord': stock_point_coord,
            'Trips': trips
        }
    
    return result


# Alternative version if you want a single dictionary (assuming only one stock point)
def create_single_stockpoint_dict(df_selected_trip, df_stockpoint_dim):
    """
    Create a single dictionary structure for one stock point.
    
    Parameters:
    df_selected_trip: DataFrame with trip data for one stock point
    df_stockpoint_dim: DataFrame with stock point dimension data
    
    Returns:
    dict: Single dictionary with stock point information and trips
    """
    if df_selected_trip.empty:
        logger.info('Dataframe is empty')
        return {}
    
    # Get the stock point ID (assuming all rows have the same stock point)
    stock_point_id = df_selected_trip['StockPointID'].iloc[0]
    
    # Get stock point information from df_stockpoint_dim
    stock_point_info = df_stockpoint_dim[df_stockpoint_dim['Stock_Point_ID'] == stock_point_id]
    
    if stock_point_info.empty:
        stock_point_name = df_selected_trip['StockPointName'].iloc[0]
        stock_point_coord = [0, 0]  # Placeholder
    else:
        stock_point_name = stock_point_info['Stock_point_Name'].iloc[0] 
        stock_point_coord = [
            stock_point_info['Longitude'].iloc[0], 
            stock_point_info['Latitude'].iloc[0]
        ]
    
    # Group by TripID
    trip_groups = df_selected_trip.groupby('TripID')
    trips = []
    
    for trip_id, trip_group in trip_groups:
        destinations = []
        for _, row in trip_group.iterrows():
            destination = {
                'CustomerID': row['CustomerID'],
                'Coordinate': [row['Longitude'], row['Latitude']]
            }
            destinations.append(destination)
        
        trip_dict = {
            'TripID': trip_id,
            'Destinations': destinations
        }
        trips.append(trip_dict)
    
    # Return the final dictionary
    return {
        'StockPointName': stock_point_name,
        'StockPointID': stock_point_id,
        'StockPointCoord': stock_point_coord,
        'Trips': trips
    }


# Example usage:
# """
# # For multiple stock points:
# result_dict = create_stockpoint_dict(df_selected_trip, df_stockpoint_dim)

# # For a single stock point:
# single_result = create_single_stockpoint_dict(df_selected_trip, df_stockpoint_dim)
# """

In [28]:
def create_route(df_selected_trip, df_stockpoint_dim):
    # Path 
    main_dir = f'./recommendation_output/selected_trip_map/{CURRENT_DATE}' 
    os.makedirs(f'{main_dir}', exist_ok=True)


    trip_dict = create_single_stockpoint_dict(df_selected_trip, df_stockpoint_dim) 

    if trip_dict == {}:
        logger.info('Trip Data is empty')
    else:
        try:
            StockPointID = trip_dict['StockPointID']
            output_filename = f'{main_dir}/{StockPointID}.html'
            # Step 1: Get route information for all trips
            calculated_routes_info = get_valhalla_routes_info(trip_dict)

            # Step 2: Plot all routes on a map
            plot_routes_on_map(trip_data=trip_dict, routes_info=calculated_routes_info, output_filename = output_filename)
        except Exception as e:
            logger.warn(f'Some vital error occured while creating route {e}')

In [29]:
def run_route_optimizer(df_clustering, sel_cluster_tuple, df_stockpoint, 
                        stock_point_name,
                        sel_total_customer_count, capacity_size = 20):
     # ---- SETUP CLIENT
     try:
          client = Valhalla(base_url=VALHALLA_BASE_URL)
          if VALHALLA_API_KEY:
               client = Valhalla(base_url=VALHALLA_BASE_URL, api_key=VALHALLA_API_KEY)
          
          logger.info('Setting up routing client via LOCAL host Valhalla')
     except Exception as e:
          logger.warning('Setting up routing client via ORS')
          client = ors.Client(key=os.getenv('ORS_KEY')) 

     # Select cluster 37
     df_sel_clust = df_clustering.query(f'cluster in {sel_cluster_tuple}').query('Latitude > 0')

     # Ensure coordinates are in [longitude, latitude] for ORS
     coords = [[lon, lat] for lat, lon in zip(df_sel_clust.Latitude, df_sel_clust.Longitude)]
     # Print number of jobs
     print("Number of customer locations:", len(coords))
     # Convert depot_location to ORS format
     # Assuming depot_location is [lat, lon], flip to [lon, lat]
     vehicle_start = [df_stockpoint.Longitude[0], df_stockpoint.Latitude[0]]
     num_vehicles = math.floor(sel_total_customer_count / capacity_size)
     vehicles = [
          ors.optimization.Vehicle(
               id=i,
               profile='driving-car',
               start=vehicle_start,
               end=vehicle_start,
               capacity=[capacity_size]
          ) for i in range(num_vehicles)
     ]

     # Define jobs (each customer gets amount=[1])
     jobs = [ors.optimization.Job(id=index, location=coord, amount=[1]) for index, coord in enumerate(coords)]

     # Call ORS optimization API
     optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

     #     ------ MAP
     depot_location = [df_stockpoint.Latitude[0], df_stockpoint.Longitude[0]]
     depot_name = df_stockpoint.Stock_point_Name[0]

     map_clusters_route = create_enhanced_cluster_map(
     df_sel_clust,
     popup_cols=['CustomerID', 'LGA', 'LCDA'],
     tooltip_cols=['LGA', 'LCDA'], 
     zoom_start=10, 
     radius=10
     ).add_child(folium.Marker(location=depot_location, 
                         size = 10, 
                         tooltip=depot_name, 
                         icon=folium.Icon(color="green", 
                         icon="home")))

     # line_colors = ['green', 'orange', 'blue', 'yellow']
     separable_colors = [
          "#1f77b4",  # blue
          "#ff7f0e",  # orange
          "#2ca02c",  # green
          "#d62728",  # red
          "#9467bd",  # purple
          "#8c564b",  # brown
          "#e377c2",  # pink
          "#7f7f7f",  # gray
          "#bcbd22",  # yellow-green
          "#17becf",  # cyan
          "#aec7e8",  # light blue
          "#ffbb78",  # light orange
          ]

     line_colors = separable_colors[0:num_vehicles] #['green', 'orange', 'blue', 'yellow']
     for route in optimized['routes']:
          folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(map_clusters_route)

     #
     selected_trip_map_path = f'./recommendation_output/selected_trip_map/{stock_point_name}_{CURRENT_DATE}.html' 
     map_clusters_route.save(selected_trip_map_path)

In [30]:
def cluster_trip_route(df_sku_rec, 
                       df_customer_dim, 
                       df_stockpoint,
                       stock_point_id,
                       max_customers_per_route, 
                       max_volume_per_route,
                       max_distance_km, 
                       clustering_method='divisive',
                       skip_route_optimization = False):
     

    optimizer = RouteOptimizer(
        max_customers_per_route=max_customers_per_route,
        max_volume_per_route=max_volume_per_route,
        max_distance_km = max_distance_km
    )

    optimizer.load_data(df_sku_rec, df_customer_dim, df_stockpoint)
    print("✓ Route optimizer initialized")

    # STEP 3: Generate Routes for Stock Point 1647113
    print("\n3. Generating Optimized Routes...")
    print("-" * 40) 

    stock_point = df_stockpoint[df_stockpoint['Stock_Point_ID'] == stock_point_id].reset_index(drop = True)
    
    stock_point_coords = (stock_point['Latitude'], stock_point['Longitude'])
        
    clustering_customers_df = optimizer.filter_customers_for_stockpoint(stock_point_id)

    df_clustering, n_clusters = optimizer.create_geographic_clusters(clustering_customers_df, 
                                                                     clustering_method = clustering_method)

    if skip_route_optimization == True:
        routes = optimizer.generate_multi_trip_routes(stock_point_id, 
                                                    max_trips=5, 
                                                    clustering_method=clustering_method)
        df_routes = pd.DataFrame(routes)
    else:
        df_routes = pd.DataFrame()



    # STEP 4: Analyze Results
    print("\n4. Route Analysis & Results...")
    print("-" * 40)

    push_recommendation = df_sku_rec.merge(df_clustering[['Stock_Point_ID','CustomerID', 'cluster']], 
                                           how='inner', on =['Stock_Point_ID','CustomerID'] )
    
    ### Cluster Evaluation
    try:
        evaluate_unsupervised_clustering(df_clustering)
    except:
        pass

    return push_recommendation, df_clustering, df_routes, stock_point_coords
    

In [31]:
def cluster_summary_and_selection(push_recommendation,
                                  sel_trip_cluster,
                                  min_ncust_per_cluster = 4
                                  ):
    ### Cluster Summary 
    cluster_summary = (
        push_recommendation
        .groupby('cluster').agg(
            LGA_list = ('LGA', lambda x: x.unique().tolist()),
            LCDA_List = ('LCDA', lambda x: x.unique().tolist()),
            ncustomer = ('CustomerID','nunique'),
            totalQty = ('EstimatedQuantity','sum'), 
            avg_customer_score = ('composite_customer_score','mean'),
        )
        .reset_index()
        .sort_values(['avg_customer_score','ncustomer', 'totalQty'], 
                     ascending=[False, False, False])
        )

    ### Select Trip   
    df_high_value_cluster_summary = (
            cluster_summary
            .query(f'ncustomer >= {min_ncust_per_cluster}')
            .head(max(10, sel_trip_cluster))
            .reset_index(drop = True)
        )
    sel_cluster_tuple = df_high_value_cluster_summary.cluster[0:sel_trip_cluster].to_list()
    sel_total_customer_count = df_high_value_cluster_summary.head(sel_trip_cluster).ncustomer.sum()
    print(f'''Select ClusterIDs: {sel_cluster_tuple}''')
    print(f'''Total Number of Customers: {sel_total_customer_count}''')
    print(df_high_value_cluster_summary.head(sel_trip_cluster))

    return cluster_summary, df_high_value_cluster_summary, sel_cluster_tuple, sel_total_customer_count

In [32]:
def prep_selected_trip(push_recommendation, 
                       cluster_summary,
                       df_master_customer_dim,  
                       df_stockpoint,
                       sel_cluster_tuple):
    
        

    sel_columns = ['Stock_Point_ID', 
                'StateName', # 'Region', 
                'Latitude', 'Longitude', 'LGA', 'LCDA', 'cluster', 
                'CustomerID', 'SKUID', 'ProductName', 'Output',
                'LastDeliveredDate', 'Recency', 'InventoryCheck', 'ProductTag', 'Medium',
                'EstimatedQuantity', 
                # 'CustomerSKUscoreRank'
                ]

    sel_cols_cust= ['Stock_Point_ID', 'CustomerID', 'ContactName',  'CustomerModeName',   'ContactPhone', 'FullAddress', 
                    'composite_customer_score', 'percentile_rank',  'KYC_Capture_Status', 'CustomerPurchaseRecency']

    final_cols = ['Stock_Point_ID', 'Stock_point_Name', 'TripID', 'LGA_list', 'LCDA_List', 
                  'ncustomer', 'totalQty','avg_customer_score', 'CustomerID', 'ContactName',  
                  'CustomerModeName',   'ContactPhone', 'FullAddress', 'Latitude',
                  'Longitude', 'LGA', 'LCDA', 'composite_customer_score', #, 'percentile_rank',  
                  'KYC_Capture_Status', 'SKUID', 'ProductName', #'Output', 'LastDeliveredDate', 
                  'Recency','CustomerPurchaseRecency', 'InventoryCheck', 'ProductTag', 'Medium', 'EstimatedQuantity',
                ]
    
    def _merge_select(df):
        modified_df = (
                        df[sel_columns]
                        .merge(cluster_summary, how='left', on = 'cluster' )
                        .merge(df_master_customer_dim[sel_cols_cust], how='left', on = ['Stock_Point_ID', 'CustomerID'])
                        .merge(df_stockpoint[['Stock_Point_ID', 'Stock_point_Name']], how='left', on = ['Stock_Point_ID'])
                        .rename(columns={'cluster':'TripID'})
                        [final_cols]
                        .rename(columns = {
                                           'Stock_point_Name': 'StockPointName'
                                           ,'Stock_Point_ID': 'StockPointID'
                                           ,'ncustomer': 'TotalCustonerCount'
                                           ,'totalQty': 'TripTotalQuantity'
                                           ,'avg_customer_score': 'TripAvgCustomerScore'
                                           ,'LastDeliveredDate': 'CustomerLastDeliveredDate'
                                           ,'Medium': 'RecommendationType'
                                           ,'Recency': 'SKUDaysSinceLastBuy'
                                           ,'CustomerPurchaseRecency': 'CustomerDaysSinceLastBuy'
                                           ,'composite_customer_score': 'CustomerScore'
                                           ,'KYC_Capture_Status': 'kycCaptureStatus'
                                           ,'LGA_list': 'ClusterLGAs'
                                           ,'LCDA_List': 'ClusterLCDAs'
                                           })
                        )
        return modified_df

    df_selected_trip = push_recommendation[push_recommendation['cluster'].isin(sel_cluster_tuple)]
    selected_push_recommendation_trip = _merge_select(df_selected_trip)
    all_push_recommendation =  _merge_select(push_recommendation)
    all_push_recommendation['isTripSelected'] = np.where(all_push_recommendation['TripID'].isin(sel_cluster_tuple) ,
                                                    'Yes',
                                                    'No'
                                                )
    

    return selected_push_recommendation_trip, all_push_recommendation

### Main Function

In [ ]:
def run_push_recommendation(df_customer_sku_recommendation, 
                            df_master_customer_dim, 
                            df_stockpoint_dim, 
                            stock_point_id,
                            stock_point_name,
                            sku_recency = 7, 
                            customer_recency = 60, number_recommendation = 5, 
                            estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                            exclude_recency_customer = 4,
                            max_customers_per_route=20,
                            max_volume_per_route=300,
                            max_distance_km = 40,
                            sel_trip_cluster = 5,
                            min_ncust_per_cluster = 5,
                            clustering_method = 'divisive',
                            skip_route_optimization = False):
    """
    Main execution function demonstrating complete route optimization workflow
    """ 

    print("=" * 80)
    print("ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS")
    print(f"StockPoint: {stock_point_name}, StockPointID: {stock_point_id},")
    print("=" * 80)

    # STEP 1: Load or Generate Data
    print("\n1. Loading Data...")
    print("-" * 40)

    df_sku_rec, df_customer_dim, df_stockpoint  = data_filter(df_customer_sku_recommendation, 
                                                                df_master_customer_dim, 
                                                                df_stockpoint_dim, 
                                                                stockpoint_id = stock_point_id,  
                                                                sku_recency = sku_recency, 
                                                                customer_recency = customer_recency, 
                                                                number_recommendation = number_recommendation,
                                                                estimate_qty_scale_factor = estimate_qty_scale_factor, 
                                                                max_estimated_qty = max_estimated_qty,
                                                                exclude_recency_customer = exclude_recency_customer)

    if len(df_customer_dim) < min_ncust_per_cluster:
        return {}
    
    print(f"✓ Loaded {len(df_sku_rec)} SKU recommendations")
    print(f"✓ Loaded {len(df_customer_dim)} customer records")
    print(f"✓ Loaded {len(df_stockpoint)} stock points")

    push_recommendation, df_clustering, df_routes, stock_point_coords = cluster_trip_route(df_sku_rec, 
                                                                                            df_customer_dim, 
                                                                                            df_stockpoint,
                                                                                            stock_point_id,
                                                                                            max_customers_per_route, 
                                                                                            max_volume_per_route,
                                                                                            max_distance_km,
                                                                                            clustering_method,
                                                                                            skip_route_optimization)

    ### Cluster Evaluation
    try:
        evaluate_unsupervised_clustering(df_clustering)
    except:
        pass
    
    ### Cluster Summary 
    cluster_summary, df_high_value_cluster_summary, sel_cluster_tuple, sel_total_customer_count = cluster_summary_and_selection(
                                                                                                        push_recommendation,
                                                                                                        sel_trip_cluster,
                                                                                                        min_ncust_per_cluster = min_ncust_per_cluster
                                                                                                        )

    ## Trip
    selected_push_recommendation_trip, all_push_recommendation = prep_selected_trip(push_recommendation, 
                                                  cluster_summary, 
                                                  df_master_customer_dim,  
                                                  df_stockpoint,
                                                  sel_cluster_tuple)
    
 
    ### Trip Maps
    if skip_route_optimization:
        try:
            df_selected_trip_summary =  selected_push_recommendation_trip.groupby(['StockPointID','TripID', 
                                                                                   'CustomerID', 'Latitude','Longitude',
                                                                                   'LGA', 'LCDA','CustomerScore']).agg( 
                        TotalQuantity = ('EstimatedQuantity','sum')
                        ,TotalSKU = ('SKUID','nunique')
                    ).reset_index()
            create_route(df_selected_trip_summary, df_stockpoint)
        except Exception as e:
            pass
    else:
        try:
            trip_map_path = f'./recommendation_output/trip_map/{stock_point_name}_{CURRENT_DATE}.html' 
            map_clusters = vis_and_save(df_routes= (df_routes
                                                .rename(columns={'cluster':'cluster_bck'})
                                                .rename(columns={'TripNumber':'cluster'})
                                                ), 
                                        df_stockpoint=df_stockpoint, 
                                        filename=trip_map_path)
        except Exception as e:
            print(f'Unable to save the generated map image: {e}')

        try:
            run_route_optimizer(df_clustering, sel_cluster_tuple, df_stockpoint, 
                            stock_point_name,
                            sel_total_customer_count, 
                            capacity_size = 20)
        except Exception as e:
            print(f'Unable to generate route mapping using orc: {e}')       
    

    ### Export Data
    try:
        export_data(
                selected_trip = selected_push_recommendation_trip,
                all_push_recommendation = all_push_recommendation,
                cluster_summary = cluster_summary,
                stock_point_name = stock_point_name,
                CURRENT_DATE = CURRENT_DATE
            )
    except Exception as e:
        print(f'Unable to generate route mapping using orc: {e}')

    dict_ = {
        'stock_point_name': stock_point_name,
        'selected_trip': selected_push_recommendation_trip,
        'all_push_recommendation': all_push_recommendation,
        'cluster_summary': cluster_summary
    }

    return dict_
    #push_recommendation, df_clustering, df_routes, trip_summary, stock_point_coords, df_stockpoint

# Usage

In [34]:
## Load Data
df_customer_sku_recommendation_raw = pd.read_feather('./input/customer_sku_recommendation.feather').rename(columns={'FCID':'Stock_Point_ID','CustomerId':'CustomerID'})
df_customer_dim_with_affinity_score_raw = pd.read_feather('./input/customer_dim_with_affinity_score.feather').rename(columns={'FCID':'Stock_Point_ID'})
df_stockpoint_dim_raw = pd.read_feather('./input/stockpoint_dim.feather')
df_kyc_customer = pd.read_feather('./input/kyc_customers.feather')
df_customer_score = pd.read_feather('./input/df_customer_score.feather')

In [35]:
# Preprocessing
df_customer_sku_recommendation, df_master_customer_dim, df_stockpoint_dim = preprocessing(df_customer_sku_recommendation_raw, 
                                                                                                        df_customer_dim_with_affinity_score_raw, 
                                                                                                        df_stockpoint_dim_raw,
                                                                                                        df_customer_score,
                                                                                                        df_kyc_customer)

In [36]:
# causeway_customer_dim.query("days_since_last_order > 60")[['KYC_Capture_Status']].value_counts()
# causeway_customer_dim.query("KYC_Capture_Status == 'No'")[['days_since_last_order']].hist()#.value_counts()#.reset_index().sort_values('days_since_last_order')
# causeway_customer_dim.KYC_Capture_Status.value_counts()

In [37]:
# [col for col in df_stockpoint_dim.Stock_point_Name if 'C' in col] 
# test_sp = 'OmniHub Apapa Lagos - CAUSEWAY'
# # # test_spid = 1647402
# # test_sp = 'OmniHub Alimosho Lagos - Barka Agro Mix'
# # test_spid = 1647345

# df_stockpoint_dim[df_stockpoint_dim['Stock_point_Name'] == test_sp]
# # # df_customer_sku_recommendation.query(f'Stock_Point_ID == {test_spid}')

### Iterative Run - All SPs

In [38]:
ALL_STOCKPOINTS_RESULT = {}
for index, row in df_stockpoint_dim.iterrows():
    # if index == 12:
    # if index == 5:
    stock_point_id =  row['Stock_Point_ID']
    stock_point_name = row['Stock_point_Name']
    print(f'{index}/{len(df_stockpoint_dim)} \nStock Point ID: {stock_point_id} || Stock Point Name: {stock_point_name}')  # Access by column name

    res_dict = run_push_recommendation(df_customer_sku_recommendation, 
                                df_master_customer_dim, 
                                df_stockpoint_dim, 
                                stock_point_id,
                                stock_point_name,
                                sku_recency = 7, 
                                customer_recency = 60, number_recommendation = 10, 
                                estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                                exclude_recency_customer = 5,
                                max_customers_per_route=20,
                                max_volume_per_route=300,
                                max_distance_km = 5,
                                sel_trip_cluster = 4,
                                min_ncust_per_cluster = 5,
                                clustering_method = 'divisive',
                                skip_route_optimization = True)
    
    ALL_STOCKPOINTS_RESULT[stock_point_name] = res_dict

0/72 
Stock Point ID: 1647128 || Stock Point Name: OmniHub Obio Akpor Rivers - Rivoc
ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
StockPoint: OmniHub Obio Akpor Rivers - Rivoc, StockPointID: 1647128,

1. Loading Data...
----------------------------------------
Total Quantity before filter: 50
Total Quantity: 0
Total Number of Customers before filter: 1
Total Number of Customers: 0
1/72 
Stock Point ID: 1647401 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Prince Tunadek
ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
StockPoint: OmniHub Ado Odo/Ota Ogun - Prince Tunadek, StockPointID: 1647401,

1. Loading Data...
----------------------------------------
Total Quantity before filter: 218
Total Quantity: 67
Total Number of Customers before filter: 19
Total Number of Customers: 4
2/72 
Stock Point ID: 1647402 || Stock Point Name: OmniHub AMAC 1 Abuja - Elriah
ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
StockPoint: OmniHub AMAC 1 Abuja - Elriah, StockPointID: 1647402,

1. Lo

INFO:__main__:Dataframe is empty
INFO:__main__:Trip Data is empty


Total Quantity before filter: 100
Total Quantity: 55
Total Number of Customers before filter: 9
Total Number of Customers: 5
✓ Loaded 16 SKU recommendations
✓ Loaded 5 customer records
✓ Loaded 1 stock points
✓ Route optimizer initialized

3. Generating Optimized Routes...
----------------------------------------

4. Route Analysis & Results...
----------------------------------------
Silhouette Score: 0.41
Davies-Bouldin Index: 0.18
Calinski-Harabasz Score: 23.69
Silhouette Score: 0.41
Davies-Bouldin Index: 0.18
Calinski-Harabasz Score: 23.69
Select ClusterIDs: []
Total Number of Customers: 0
Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
3/72 
Stock Point ID: 1647136 || Stock Point Name: OmniHub Tarauni Kano - Amjabil
ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
StockPoint: OmniHub Tarauni Kano - Amjabil, StockPointID: 1647136,

1. Loading Data...
----------------------------------------
Total Quantity before filter: 5


INFO:__main__:Dataframe is empty
INFO:__main__:Trip Data is empty


45/72 
Stock Point ID: 1647380 || Stock Point Name: OmniHub Owerri Municipal Imo - Bonaventure 
ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
StockPoint: OmniHub Owerri Municipal Imo - Bonaventure , StockPointID: 1647380,

1. Loading Data...
----------------------------------------
Total Quantity before filter: 205
Total Quantity: 67
Total Number of Customers before filter: 18
Total Number of Customers: 8
✓ Loaded 14 SKU recommendations
✓ Loaded 8 customer records
✓ Loaded 1 stock points
✓ Route optimizer initialized

3. Generating Optimized Routes...
----------------------------------------

4. Route Analysis & Results...
----------------------------------------
Silhouette Score: 0.42
Davies-Bouldin Index: 0.31
Calinski-Harabasz Score: 55.48
Silhouette Score: 0.42
Davies-Bouldin Index: 0.31
Calinski-Harabasz Score: 55.48
Select ClusterIDs: []
Total Number of Customers: 0
Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
46/7

In [ ]:
print(res_dict.keys())
# [col  for col in res_dict['all_push_recommendation'].columns if 'KYC_Capture_Status' in col]
# res_dict['cluster_summary'].head(10)

## Routing 2

In [ ]:
df_test = ALL_STOCKPOINTS_RESULT['OmniHub Apapa Lagos - CAUSEWAY'] 
df_selected_trip = df_test['selected_trip']
print(df_selected_trip.TripID.nunique())
# dict_keys(['stock_point_name', 'selected_trip', 'all_push_recommendation', 'cluster_summary'])

In [ ]:
df_selected_trip_summary =  df_selected_trip.groupby(['StockPointID','TripID', 
                                                                                   'CustomerID', 'Latitude','Longitude',
                                                                                   'LGA', 'LCDA','CustomerScore']).agg( 
                        TotalQuantity = ('EstimatedQuantity','sum')
                        ,TotalSKU = ('SKUID','nunique')
                    ).reset_index() 
# trip_dict = create_single_stockpoint_dict(df_selected_trip_summary, df_stockpoint_dim) 
# route_info = calculated_routes_info = get_valhalla_routes_info(trip_dict)
# route_info[0].keys()
# len(trip_dict['Trips'][0]['Destinations'])
create_route(df_selected_trip_summary, df_stockpoint_dim)

In [ ]:
create_route(df_selected_trip_summary, df_stockpoint_dim)

# Data Export to DB

In [ ]:
# ALL_STOCKPOINTS_RESULT.keys()
# # ALL_STOCKPOINTS_RESULT['OmniHub Obio Akpor Rivers - Rivoc']
# ALL_STOCKPOINTS_RESULT['OmniHub Ado Odo/Ota Ogun - Prince Tunadek']

In [39]:
DF_ALL_RECOMMENDATION = pd.DataFrame() 
for key in ALL_STOCKPOINTS_RESULT.keys():
    dict_f = ALL_STOCKPOINTS_RESULT[key]
    if dict_f != {}:
        df_ = dict_f['all_push_recommendation']
    
        if len(df_) > 0:
            df_['ClusterLGAs'] = df_['ClusterLGAs'].apply(str)
            df_['ClusterLCDAs'] = df_['ClusterLCDAs'].apply(str)
            DF_ALL_RECOMMENDATION = pd.concat([DF_ALL_RECOMMENDATION, df_])
            DF_ALL_RECOMMENDATION['ModifiedDate'] = CURRENT_DATE

In [40]:
cols_summary = ['StockPointID', 'StockPointName', 'TripID', 'ClusterLGAs', 'ClusterLCDAs', 'TotalCustonerCount', 'TripTotalQuantity','TripAvgCustomerScore', 'ModifiedDate']  
DF_CLUSTER_SUMMARY = DF_ALL_RECOMMENDATION[cols_summary].drop_duplicates().reset_index(drop=True)

In [41]:
pd.set_option('display.max_columns', None)
# DF_ALL_RECOMMENDATION.sample(1)
DF_CLUSTER_SUMMARY.sample(1)

,StockPointID,StockPointName,TripID,ClusterLGAs,ClusterLCDAs,TotalCustonerCount,TripTotalQuantity,TripAvgCustomerScore,ModifiedDate
245,1647372,OmniHub AMAC 1 Abuja - Roekwi,8,['Kuje/Gwagwalada/Abaji'],"['ABUJA- ZUBA', 'ABUJA-GWAGWALADA']",3,28,20.015,2025-06-28


In [ ]:
# max_lengths = DF_ALL_RECOMMENDATION.astype(str).applymap(len).max().reset_index(name = 'max_length')
# print(max_lengths)
# print(DF_ALL_RECOMMENDATION.info())

In [42]:
def upsert_dataframe(df, table_name, conn, match_cols, update_cols, batch_size = 2000, fast_executemany = True):
    # Input validation
    if df.empty:
        raise ValueError("DataFrame is empty.")
    if not match_cols or not update_cols:
        raise ValueError("match_cols and update_cols cannot be empty.")
    if not all(col in df.columns for col in match_cols + update_cols):
        raise ValueError("Some match_cols or update_cols are not in the DataFrame.")
    if not table_name.strip() or any(c in table_name for c in ".;[]"):
        raise ValueError("Invalid table_name.")

    cursor = None
    try:
        cursor = conn.cursor()
        cursor.fast_executemany = fast_executemany 

        staging_table = f"#{table_name}_staging"
        cols = df.columns.tolist()
        col_list = ', '.join(f"[{col}]" for col in cols)
        placeholders = ', '.join(['?'] * len(cols))

        # Step 1: Create staging table from real schema
        create_staging_sql = f"""
        SELECT TOP 0 {col_list}
        INTO {staging_table}
        FROM {table_name}
        WHERE 1 = 0;
        """
        cursor.execute(create_staging_sql)

        # Step 2: Bulk insert into staging table using fast_executemany
        insert_sql = f"INSERT INTO {staging_table} ({col_list}) VALUES ({placeholders})"
        cursor.executemany(insert_sql, df[cols].values.tolist())
        conn.commit()

        # insert_sql = f"INSERT INTO {staging_table} ({col_list}) VALUES ({placeholders})"
        # data = df[cols].values.tolist()
        # for i in range(0, len(data), batch_size):
        #     cursor.executemany(insert_sql, data[i:i+batch_size])
        # conn.commit()

        # Step 3: MERGE for upsert
        on_clause = ' AND '.join([f"TARGET.[{col}] = SOURCE.[{col}]" for col in match_cols])
        update_clause = ', '.join([f"TARGET.[{col}] = SOURCE.[{col}]" for col in update_cols])
        insert_cols = ', '.join([f"[{col}]" for col in cols])
        insert_values = ', '.join([f"SOURCE.[{col}]" for col in cols])

        merge_sql = f"""
        MERGE {table_name} AS TARGET
        USING {staging_table} AS SOURCE
        ON {on_clause}
        WHEN MATCHED THEN
            UPDATE SET {update_clause}
        WHEN NOT MATCHED THEN
            INSERT ({insert_cols})
            VALUES ({insert_values});
        """
        cursor.execute(merge_sql)
        conn.commit()
    except Exception as e:
        if conn:
            conn.rollback()
        logger.error(f"Upsert failed for table {table_name}: {e}")
        raise Exception(f"Upsert failed for table {table_name}: {e}") from e
    finally:
        if cursor:
            cursor.close()


In [43]:
df_insert = DF_ALL_RECOMMENDATION.drop(columns=['ClusterLGAs',	'ClusterLCDAs']).reset_index(drop=True)
# Replace 'YourProblematicColumn' with the actual column name you found
df_insert['SKUDaysSinceLastBuy'] = df_insert['SKUDaysSinceLastBuy'].astype(float, errors='ignore').astype(int, errors='ignore')
df_insert['CustomerDaysSinceLastBuy'] = df_insert['CustomerDaysSinceLastBuy'].astype(float, errors='ignore').astype(int, errors='ignore')

match_cols = ['StockPointID', 'CustomerID', 'SKUID', 'ModifiedDate']
update_cols = list(set(df_insert.columns) - set(match_cols))

conn = get_connection()
upsert_dataframe(
    df=df_insert,
    table_name='dailyPredictedPull',
    conn=conn,
    match_cols=match_cols,
    update_cols=update_cols   
) 
 
print(conn.closed)
conn.close()
print(conn.closed)

False
True


In [44]:
df = DF_CLUSTER_SUMMARY.copy() 

df['ClusterLGAs'] = df['ClusterLGAs'].astype(str).str.slice(0, 500)
df['ClusterLCDAs'] = df['ClusterLCDAs'].astype(str).str.slice(0, 500)


match_cols = ['StockPointID', 'TripID', 'ModifiedDate']
update_cols = list(set(df.columns) - set(match_cols))

conn = get_connection()
upsert_dataframe(
    df=df,
    table_name='dailyPredictedPullClusterSummary',
    conn=conn,
    match_cols=match_cols,
    update_cols=update_cols ,
    fast_executemany = False  
)

print(conn.closed)
conn.close()
print(conn.closed)

False
True


In [ ]:
# conn.dispose()

In [ ]:
# # pd.set_option('display.width', None)
# pd.set_option('display.max_columns', None)
# res_dict['all_push_recommendation'].sample(1)
# # res_dict['cluster_summary'].sort_values('ncustomer', ascending = False)
# from collections import Counter
# Counter(res_dict['all_push_recommendation'].columns)

# Case Test

In [ ]:
# Data Filter - Testing 
causeway, causeway_customer_dim, causeway_stockpoint, = data_filter(df_customer_sku_recommendation, 
                                                                    df_master_customer_dim, 
                                                                    df_stockpoint_dim, 
                                                                    stockpoint_id = 1647394,  
                                                                    # stockpoint_id = 1647113,  
                                                                    sku_recency = 7, customer_recency = 60, number_recommendation = 10,
                                                                    estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                                                                    exclude_recency_customer = 4)

# Total Number of Customers: 905 || 901

df_all_cluster = causeway_customer_dim[['CustomerID', 'Latitude','Longitude']].drop_duplicates()
df_all_cluster.shape
# df_all_cluster = res_dict['all_push_recommendation'][['CustomerID','TripID', 'Latitude','Longitude']].drop_duplicates().rename(columns={'TripID':'cluster'})


# vis_and_save(df_routes = df_all_cluster,
#                  df_stockpoint = None,   
#                  filename=None,
#                  cluster_col = 'cluster')

# 459
# 364

In [ ]:
# df_all_cluster.Latitude.describe()

In [ ]:
# cols = ['CustomerID', 'SKUID', 'Medium','CustomerPurchaseRecency']
# causeway[cols].query('(CustomerID == 5271729) or (CustomerID ==  5266873)')
# causeway.groupby(['CustomerPurchaseRecency'])['CustomerID'].nunique().reset_index().sort_values('CustomerID')

In [ ]:
# causeway.columns
# causeway['Medium'].value_counts()

### Test New Clustering

In [ ]:
from clustering.divisive_clustering import DivisiveGeographicClustering, OptimizedDivisiveGeographicClustering

print("\n" + "="*60)
print("2. DIVISIVE HIERARCHICAL CLUSTERING")
print("="*60)

divisive_clusterer = OptimizedDivisiveGeographicClustering( 
    # Rivers: Divisive clusters created: 48 || Silhouette Score: 0.54 || Constraint violations: Size=6, Distance=3
    max_customers_per_cluster=30,  # REQUIRED
    max_distance_km=5            # REQUIRED
    ,use_vectorized_distances=True, balance_clusters=False
)

divisive_result = divisive_clusterer.divisive_clustering(df_all_cluster.copy())
print(f"\nDivisive clusters created: {divisive_result['cluster'].nunique()}")
# print(f"Cluster sizes: {divisive_result['cluster'].value_counts().sort_index().head()}")
_ = evaluate_unsupervised_clustering(divisive_result)

# # Get detailed statistics
stats = divisive_clusterer.get_cluster_stats(divisive_result)
print(f"Total clusters: {stats['summary']['total_clusters']}")
print(f"Constraint violations: Size={stats['summary']['size_violations']}, Distance={stats['summary']['distance_violations']}")



In [ ]:
from clustering.agglomerative_clustering import AgglomerativeGeographicClustering
print("\n" + "="*60)
print("2. AGGLOMERATIVE CLUSTERING")
print("="*60)
 
agg_clusterer = AgglomerativeGeographicClustering(
    max_customers_per_cluster=30, # Aim for clusters of max 50 customers
    max_distance_km=5.0,        # Max diameter of 5 km
    linkage_method='ward',       # Common choice for compact clusters
    sub_cluster_if_oversized=True
)
clustered_agg_df = agg_clusterer.agglomerative_clustering(df_all_cluster.copy())

# print("\nAgglomerative Clustering Stats:")
# for k, v in agg_stats['summary'].items():
#     print(f"  {k}: {v}")
# print("\nSample Agglomerative Cluster Details:")
# for cluster_id, details in list(agg_stats.items())[:5]: # Show first 5 clusters
#     if cluster_id != 'summary':
#         print(f"  Cluster {cluster_id}: Size={details['size']}, Diameter={details['diameter_km']:.2f} km, Size OK={details['meets_size_constraint']}, Distance OK={details['meets_distance_constraint']}")


_ = evaluate_unsupervised_clustering(clustered_agg_df)


stats = agg_clusterer.get_cluster_stats(clustered_agg_df, agg_clusterer.max_customers_per_cluster, agg_clusterer.max_distance_km) 
print(f"Total clusters: {stats['summary']['total_clusters']}")
print(f"Constraint violations: Size={stats['summary']['size_violations']}, Distance={stats['summary']['distance_violations']}")


In [ ]:
vis_and_save(df_routes = clustered_agg_df,
                 df_stockpoint = None,   
                 filename='agglomerative-clustering-test.html',
                 cluster_col = 'cluster')

In [ ]:
vis_and_save(df_routes = divisive_result,
                 df_stockpoint = None,   
                 filename='divisive-clustering-test.html',
                 cluster_col = 'cluster')

### New Clustering Algos

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

## Best
class OptimizedDivisiveGeographicClustering_b:
    def __init__(self, max_customers_per_cluster=20, max_distance_km=50, 
                 use_vectorized_distances=True, balance_clusters=False):
        self.max_customers_per_cluster = max_customers_per_cluster
        self.max_distance_km = max_distance_km
        self.earth_radius_km = 6371.0
        self.use_vectorized_distances = use_vectorized_distances
        self.balance_clusters = balance_clusters
        
    def haversine_vectorized(self, coords1, coords2=None):
        """
        Highly optimized vectorized haversine distance calculation.
        If coords2 is None, calculates pairwise distances within coords1.
        """
        if coords2 is None:
            # Pairwise distances within coords1
            coords1_rad = np.radians(coords1)
            n = len(coords1)
            
            # Create meshgrids for vectorized calculation
            lat1 = coords1_rad[:, 0]
            lon1 = coords1_rad[:, 1]
            
            lat1_mesh, lat2_mesh = np.meshgrid(lat1, lat1)
            lon1_mesh, lon2_mesh = np.meshgrid(lon1, lon1)
            
            dlat = lat2_mesh - lat1_mesh
            dlon = lon2_mesh - lon1_mesh
            
            a = (np.sin(dlat / 2) ** 2 + 
                 np.cos(lat1_mesh) * np.cos(lat2_mesh) * np.sin(dlon / 2) ** 2)
            c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
            
            return self.earth_radius_km * c
        else:
            # Distance from each point in coords1 to each point in coords2
            coords1_rad = np.radians(coords1)
            coords2_rad = np.radians(coords2)
            
            lat1 = coords1_rad[:, 0][:, np.newaxis]
            lon1 = coords1_rad[:, 1][:, np.newaxis]
            lat2 = coords2_rad[:, 0][np.newaxis, :]
            lon2 = coords2_rad[:, 1][np.newaxis, :]
            
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            
            a = (np.sin(dlat / 2) ** 2 + 
                 np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
            c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
            
            return self.earth_radius_km * c
    
    def haversine_pdist(self, coords):
        """
        Optimized haversine distance calculation using scipy's pdist.
        """
        def haversine_metric(u, v):
            lat1, lon1 = np.radians(u)
            lat2, lon2 = np.radians(v)
            
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            
            a = (np.sin(dlat / 2) ** 2 + 
                 np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
            c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
            
            return self.earth_radius_km * c
        
        return pdist(coords, metric=haversine_metric)
    
    def calculate_cluster_diameter_fast(self, coords):
        """
        Fast cluster diameter calculation with multiple optimization strategies.
        """
        n_points = len(coords)
        
        if n_points <= 1:
            return 0
        
        if n_points == 2:
            return self.haversine_single_pair(coords[0], coords[1])
        
        # Use different strategies based on cluster size
        if n_points <= 10:
            # Small clusters: exact calculation
            distances = self.haversine_pdist(coords)
            return np.max(distances)
        elif n_points <= 50:
            # Medium clusters: vectorized calculation
            if self.use_vectorized_distances:
                distance_matrix = self.haversine_vectorized(coords)
                return np.max(distance_matrix)
            else:
                distances = self.haversine_pdist(coords)
                return np.max(distances)
        else:
            # Large clusters: smart sampling
            return self._smart_diameter_estimation(coords)
    
    def _smart_diameter_estimation(self, coords):
        """
        Improved diameter estimation using multiple sampling strategies.
        """
        n_points = len(coords)
        
        # Strategy 1: Convex hull approximation
        hull_diameter = self._convex_hull_diameter(coords)
        
        # Strategy 2: Grid-based sampling for large clusters
        if n_points > 200:
            grid_diameter = self._grid_based_diameter(coords)
            return max(hull_diameter, grid_diameter)
        
        return hull_diameter
    
    def _convex_hull_diameter(self, coords):
        """Enhanced convex hull approximation."""
        lats, lons = coords[:, 0], coords[:, 1]
        
        # Get extreme points
        extreme_indices = [
            np.argmax(lats), np.argmin(lats),
            np.argmax(lons), np.argmin(lons)
        ]
        
        # Add points from different quadrants
        lat_center, lon_center = np.mean(lats), np.mean(lons)
        
        quadrants = [
            (lats >= lat_center) & (lons >= lon_center),  # NE
            (lats >= lat_center) & (lons < lon_center),   # NW
            (lats < lat_center) & (lons >= lon_center),   # SE
            (lats < lat_center) & (lons < lon_center)     # SW
        ]
        
        for quadrant in quadrants:
            if np.any(quadrant):
                quad_indices = np.where(quadrant)[0]
                # Add furthest point from center in each quadrant
                distances_from_center = np.sqrt(
                    (lats[quad_indices] - lat_center)**2 + 
                    (lons[quad_indices] - lon_center)**2
                )
                furthest_idx = quad_indices[np.argmax(distances_from_center)]
                extreme_indices.append(furthest_idx)
        
        # Add some random points
        n_random = min(12, len(coords) - len(set(extreme_indices)))
        if n_random > 0:
            available_indices = list(set(range(len(coords))) - set(extreme_indices))
            if available_indices:
                random_indices = np.random.choice(available_indices, 
                                                min(n_random, len(available_indices)), 
                                                replace=False)
                extreme_indices.extend(random_indices)
        
        # Get unique sample
        sample_indices = list(set(extreme_indices))
        sample_coords = coords[sample_indices]
        
        if len(sample_coords) <= 1:
            return 0
        
        distances = self.haversine_pdist(sample_coords)
        return np.max(distances)
    
    def _grid_based_diameter(self, coords):
        """Grid-based sampling for very large clusters."""
        # Create a grid and sample points from each grid cell
        lats, lons = coords[:, 0], coords[:, 1]
        
        # Create 6x6 grid
        lat_bins = np.linspace(lats.min(), lats.max(), 7)
        lon_bins = np.linspace(lons.min(), lons.max(), 7)
        
        sample_indices = []
        for i in range(len(lat_bins)-1):
            for j in range(len(lon_bins)-1):
                mask = ((lats >= lat_bins[i]) & (lats < lat_bins[i+1]) & 
                       (lons >= lon_bins[j]) & (lons < lon_bins[j+1]))
                cell_indices = np.where(mask)[0]
                if len(cell_indices) > 0:
                    # Sample up to 2 points from each cell
                    n_sample = min(2, len(cell_indices))
                    sampled = np.random.choice(cell_indices, n_sample, replace=False)
                    sample_indices.extend(sampled)
        
        if len(sample_indices) <= 1:
            return 0
        
        sample_coords = coords[sample_indices]
        distances = self.haversine_pdist(sample_coords)
        return np.max(distances)
    
    def haversine_single_pair(self, coord1, coord2):
        """Calculate haversine distance between two points."""
        lat1, lon1 = np.radians(coord1)
        lat2, lon2 = np.radians(coord2)
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = (np.sin(dlat / 2) ** 2 + 
             np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
        c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
        
        return self.earth_radius_km * c
    
    def should_split_cluster(self, cluster_indices, coords_array):
        """Enhanced cluster splitting logic with load balancing."""
        cluster_size = len(cluster_indices)
        
        if cluster_size <= 2:
            return False
        
        # Hard size constraint
        if cluster_size > self.max_customers_per_cluster * 1.5:
            return True
        
        # Soft size constraint with diameter check
        if cluster_size > self.max_customers_per_cluster:
            cluster_coords = coords_array[cluster_indices]
            diameter = self.calculate_cluster_diameter_fast(cluster_coords)
            return diameter > self.max_distance_km * 0.8  # More lenient for size
        
        # Diameter constraint
        cluster_coords = coords_array[cluster_indices]
        diameter = self.calculate_cluster_diameter_fast(cluster_coords)
        
        return diameter > self.max_distance_km
    
    def geographic_split(self, cluster_indices, coords_array):
        """
        Improved geographic splitting with better load balancing.
        """
        if len(cluster_indices) <= 2:
            return [cluster_indices]
        
        cluster_coords = coords_array[cluster_indices]
        n_points = len(cluster_coords)
        
        # For small clusters, use exact method
        if n_points <= 50:
            return self._exact_geographic_split(cluster_indices, cluster_coords)
        
        # For medium clusters, use K-means with geographic initialization
        if n_points <= 200:
            return self._kmeans_geographic_split(cluster_indices, cluster_coords)
        
        # For large clusters, use hierarchical approach
        return self._hierarchical_geographic_split(cluster_indices, cluster_coords)
    
    def _exact_geographic_split(self, cluster_indices, cluster_coords):
        """Exact splitting for small clusters."""
        n_points = len(cluster_coords)
        
        # Find the two points that are farthest apart
        distances = self.haversine_pdist(cluster_coords)
        distance_matrix = squareform(distances)
        max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
        center1_idx, center2_idx = max_idx[0], max_idx[1]
        
        center1 = cluster_coords[center1_idx]
        center2 = cluster_coords[center2_idx]
        
        # Assign points to closest center
        distances_to_center1 = self.haversine_vectorized(cluster_coords, center1.reshape(1, -1))[:, 0]
        distances_to_center2 = self.haversine_vectorized(cluster_coords, center2.reshape(1, -1))[:, 0]
        
        labels = (distances_to_center1 <= distances_to_center2).astype(int)
        
        return self._balance_split(cluster_indices, labels)
    
    def _kmeans_geographic_split(self, cluster_indices, cluster_coords):
        """K-means splitting with geographic initialization."""
        n_points = len(cluster_coords)
        
        # Initialize with farthest pair
        center1_idx, center2_idx = self._find_approximate_farthest_pair(cluster_coords)
        initial_centers = cluster_coords[[center1_idx, center2_idx]]
        
        # Apply K-means
        kmeans = KMeans(n_clusters=2, init=initial_centers, n_init=1, random_state=42)
        labels = kmeans.fit_predict(cluster_coords)
        
        return self._balance_split(cluster_indices, labels)
    
    def _hierarchical_geographic_split(self, cluster_indices, cluster_coords):
        """Hierarchical splitting for large clusters."""
        # Use linkage-based clustering for very large clusters
        n_sample = min(100, len(cluster_coords))
        sample_indices = np.random.choice(len(cluster_coords), n_sample, replace=False)
        sample_coords = cluster_coords[sample_indices]
        
        # Compute linkage on sample
        distances = self.haversine_pdist(sample_coords)
        linkage_matrix = linkage(distances, method='ward')
        sample_labels = fcluster(linkage_matrix, 2, criterion='maxclust') - 1
        
        # Assign all points based on closest sample point
        center1_coords = sample_coords[sample_labels == 0]
        center2_coords = sample_coords[sample_labels == 1]
        
        if len(center1_coords) == 0 or len(center2_coords) == 0:
            # Fallback to farthest pair method
            return self._exact_geographic_split(cluster_indices, cluster_coords)
        
        center1 = np.mean(center1_coords, axis=0)
        center2 = np.mean(center2_coords, axis=0)
        
        distances_to_center1 = self.haversine_vectorized(cluster_coords, center1.reshape(1, -1))[:, 0]
        distances_to_center2 = self.haversine_vectorized(cluster_coords, center2.reshape(1, -1))[:, 0]
        
        labels = (distances_to_center1 <= distances_to_center2).astype(int)
        
        return self._balance_split(cluster_indices, labels)
    
    def _balance_split(self, cluster_indices, labels):
        """Balance the split to avoid very uneven clusters."""
        cluster_0_indices = cluster_indices[labels == 0]
        cluster_1_indices = cluster_indices[labels == 1]
        
        # Ensure no empty clusters
        if len(cluster_0_indices) == 0:
            cluster_0_indices = np.array([cluster_1_indices[0]])
            cluster_1_indices = cluster_1_indices[1:]
        elif len(cluster_1_indices) == 0:
            cluster_1_indices = np.array([cluster_0_indices[0]])
            cluster_0_indices = cluster_0_indices[1:]
        
        # Optional: Balance cluster sizes if one is much larger
        if self.balance_clusters:
            size_0, size_1 = len(cluster_0_indices), len(cluster_1_indices)
            if size_0 > 3 * size_1 and size_1 > 0:
                # Move some points from cluster 0 to cluster 1
                n_move = (size_0 - size_1) // 4
                move_indices = cluster_0_indices[:n_move]
                cluster_0_indices = cluster_0_indices[n_move:]
                cluster_1_indices = np.concatenate([cluster_1_indices, move_indices])
            elif size_1 > 3 * size_0 and size_0 > 0:
                # Move some points from cluster 1 to cluster 0
                n_move = (size_1 - size_0) // 4
                move_indices = cluster_1_indices[:n_move]
                cluster_1_indices = cluster_1_indices[n_move:]
                cluster_0_indices = np.concatenate([cluster_0_indices, move_indices])
        
        return [cluster_0_indices, cluster_1_indices]
    
    def _find_approximate_farthest_pair(self, coords):
        """Find approximate farthest pair for large clusters."""
        n_points = len(coords)
        
        if n_points <= 100:
            # For moderate sizes, use exact calculation
            distances = self.haversine_pdist(coords)
            distance_matrix = squareform(distances)
            max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
            return max_idx[0], max_idx[1]
        
        # For large clusters, use sampling
        sample_size = min(50, n_points)
        sample_indices = np.random.choice(n_points, sample_size, replace=False)
        sample_coords = coords[sample_indices]
        
        distances = self.haversine_pdist(sample_coords)
        distance_matrix = squareform(distances)
        max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
        
        return sample_indices[max_idx[0]], sample_indices[max_idx[1]]
    
    def divisive_clustering(self, customers_df):
        """Perform optimized divisive hierarchical clustering."""
        customers_df = customers_df.copy().reset_index(drop=True)
        
        # Validate input
        if 'Latitude' not in customers_df.columns or 'Longitude' not in customers_df.columns:
            raise ValueError("DataFrame must contain 'Latitude' and 'Longitude' columns")
        
        coords_array = customers_df[['Latitude', 'Longitude']].values
        n_customers = len(customers_df)
        
        if n_customers == 0:
            customers_df['cluster'] = []
            return customers_df
        
        if n_customers == 1:
            customers_df['cluster'] = 1
            return customers_df
        
        # Priority queue approach for better clustering
        clusters_to_process = [(n_customers, np.arange(n_customers))]  # (size, indices)
        final_clusters = []
        
        iteration_count = 0
        max_iterations = n_customers * 2
        
        while clusters_to_process and iteration_count < max_iterations:
            # Process largest cluster first
            clusters_to_process.sort(key=lambda x: x[0], reverse=True)
            current_size, current_cluster_indices = clusters_to_process.pop(0)
            iteration_count += 1
            
            if self.should_split_cluster(current_cluster_indices, coords_array):
                subclusters = self.geographic_split(current_cluster_indices, coords_array)
                
                for subcluster_indices in subclusters:
                    if len(subcluster_indices) > 0:
                        clusters_to_process.append((len(subcluster_indices), subcluster_indices))
            else:
                final_clusters.append(current_cluster_indices)
        
        # Handle remaining clusters
        final_clusters.extend([indices for _, indices in clusters_to_process])
        
        # Create result DataFrame
        result_df = customers_df.copy()
        result_df['cluster'] = -1
        
        for cluster_id, cluster_indices in enumerate(final_clusters, 1):
            result_df.loc[cluster_indices, 'cluster'] = cluster_id
        
        return result_df
    
    def get_cluster_stats(self, clustered_df):
        """Get comprehensive clustering statistics."""
        if 'cluster' not in clustered_df.columns:
            raise ValueError("DataFrame must contain 'cluster' column")
        
        stats = {}
        cluster_sizes = []
        cluster_diameters = []
        
        for cluster_id in clustered_df['cluster'].unique():
            if cluster_id == -1:
                continue
                
            cluster_data = clustered_df[clustered_df['cluster'] == cluster_id]
            coords = cluster_data[['Latitude', 'Longitude']].values
            
            diameter = self.calculate_cluster_diameter_fast(coords)
            cluster_sizes.append(len(cluster_data))
            cluster_diameters.append(diameter)
            
            stats[cluster_id] = {
                'size': len(cluster_data),
                'diameter_km': diameter,
                'centroid_lat': np.mean(coords[:, 0]),
                'centroid_lon': np.mean(coords[:, 1]),
                'meets_size_constraint': len(cluster_data) <= self.max_customers_per_cluster,
                'meets_distance_constraint': diameter <= self.max_distance_km
            }
        
        # Overall statistics
        stats['summary'] = {
            'total_clusters': len(stats) - 1,  # Excluding summary
            'avg_cluster_size': np.mean(cluster_sizes),
            'max_cluster_size': np.max(cluster_sizes),
            'min_cluster_size': np.min(cluster_sizes),
            'avg_diameter': np.mean(cluster_diameters),
            'max_diameter': np.max(cluster_diameters),
            'size_violations': sum(1 for size in cluster_sizes if size > self.max_customers_per_cluster),
            'distance_violations': sum(1 for diameter in cluster_diameters if diameter > self.max_distance_km)
        }
        
        return stats

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.cluster import KMeans, DBSCAN
from numba import jit, prange
import warnings
warnings.filterwarnings('ignore')

# Fixed Enhanced Geographic Clustering
class OptimizedDivisiveGeographicClustering_be:
    def __init__(self, max_customers_per_cluster=20, max_distance_km=50, 
                 use_vectorized_distances=True, balance_clusters=False):
        self.max_customers_per_cluster = max_customers_per_cluster
        self.max_distance_km = max_distance_km
        self.earth_radius_km = 6371.0
        self.use_vectorized_distances = use_vectorized_distances
        self.balance_clusters = balance_clusters
        self._distance_cache = {}
        
    @staticmethod
    @jit(nopython=True, fastmath=True, parallel=True)
    def numba_haversine(coords1, coords2, radius):
        """JIT-optimized haversine distance calculation"""
        n1 = coords1.shape[0]
        n2 = coords2.shape[0] if coords2 is not None else n1
        dists = np.empty((n1, n2), dtype=np.float64)
        
        for i in prange(n1):
            lat1 = np.radians(coords1[i, 0])
            lon1 = np.radians(coords1[i, 1])
            
            for j in range(n2):
                if coords2 is None:
                    lat2 = np.radians(coords1[j, 0])
                    lon2 = np.radians(coords1[j, 1])
                else:
                    lat2 = np.radians(coords2[j, 0])
                    lon2 = np.radians(coords2[j, 1])
                    
                dlat = lat2 - lat1
                dlon = lon2 - lon1
                
                a = (np.sin(dlat/2)**2 + 
                     np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2)
                c = 2 * np.arcsin(np.sqrt(min(1.0, a)))
                dists[i, j] = radius * c
                
        return dists

    def haversine_vectorized(self, coords1, coords2=None, use_cache=False):
        """Optimized distance calculation with caching and Numba acceleration"""
        cache_key = None
        if use_cache and coords2 is None:
            cache_key = tuple(map(tuple, coords1))
            if cache_key in self._distance_cache:
                return self._distance_cache[cache_key]
        
        if coords1.size < 500:  # Use Numba for smaller datasets
            result = self.numba_haversine(coords1, coords2, self.earth_radius_km)
        else:
            # Use vectorized calculation for larger datasets
            if coords2 is None:
                coords_rad = np.radians(coords1)
                lat = coords_rad[:, 0]
                lon = coords_rad[:, 1]
                dlat = lat[:, None] - lat
                dlon = lon[:, None] - lon
                
                a = (np.sin(dlat/2)**2 + 
                     np.cos(lat[:, None]) * np.cos(lat) * np.sin(dlon/2)**2)
            else:
                coords1_rad = np.radians(coords1)
                coords2_rad = np.radians(coords2)
                dlat = coords1_rad[:, 0, None] - coords2_rad[:, 0]
                dlon = coords1_rad[:, 1, None] - coords2_rad[:, 1]
                
                a = (np.sin(dlat/2)**2 + 
                     np.cos(coords1_rad[:, 0, None]) * 
                     np.cos(coords2_rad[:, 0]) * 
                     np.sin(dlon/2)**2)
            
            result = self.earth_radius_km * 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
        
        if cache_key is not None:
            self._distance_cache[cache_key] = result
            
        return result
    
    def haversine_single_pair(self, coord1, coord2):
        """Calculate haversine distance between two points."""
        lat1, lon1 = np.radians(coord1)
        lat2, lon2 = np.radians(coord2)
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = (np.sin(dlat / 2) ** 2 + 
             np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
        c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
        
        return self.earth_radius_km * c 

    def haversine_pdist(self, coords):
        """Calculate pairwise distances using scipy format"""
        n = len(coords)
        distances = []
        for i in range(n):
            for j in range(i + 1, n):
                dist = self.haversine_single_pair(coords[i], coords[j])
                distances.append(dist)
        return np.array(distances)

    def calculate_cluster_diameter_fast(self, coords):
        """Optimized diameter calculation with adaptive strategy"""
        n = len(coords)
        if n <= 1: 
            return 0
        if n == 2:
            return self.haversine_single_pair(coords[0], coords[1])
        
        # Use convex hull approximation for medium clusters
        if n <= 100:
            return self._convex_hull_diameter(coords)
            
        # Use grid-based sampling for large clusters
        return self._grid_based_diameter(coords)

    def _convex_hull_diameter(self, coords):
        """Calculate diameter using convex hull approximation"""
        try:
            from scipy.spatial import ConvexHull
            if len(coords) < 3:
                return max(self.haversine_single_pair(coords[i], coords[j]) 
                          for i in range(len(coords)) for j in range(i+1, len(coords)))
            
            hull = ConvexHull(coords)
            hull_points = coords[hull.vertices]
            
            max_distance = 0
            for i in range(len(hull_points)):
                for j in range(i + 1, len(hull_points)):
                    distance = self.haversine_single_pair(hull_points[i], hull_points[j])
                    max_distance = max(max_distance, distance)
            
            return max_distance
        except:
            # Fallback to brute force for small clusters
            return self._brute_force_diameter(coords)

    def _grid_based_diameter(self, coords):
        """Use grid-based sampling for large clusters"""
        n_sample = min(50, len(coords))
        sample_indices = np.random.choice(len(coords), n_sample, replace=False)
        sample_coords = coords[sample_indices]
        return self._brute_force_diameter(sample_coords)

    def _brute_force_diameter(self, coords):
        """Calculate exact diameter using brute force"""
        max_distance = 0
        n = len(coords)
        for i in range(n):
            for j in range(i + 1, n):
                distance = self.haversine_single_pair(coords[i], coords[j])
                max_distance = max(max_distance, distance)
        return max_distance

    def should_split_cluster(self, cluster_indices, coords_array):
        """Determine if a cluster should be split"""
        n_points = len(cluster_indices)
        
        # Size constraint
        if n_points <= self.max_customers_per_cluster:
            return False
        
        # Distance constraint
        cluster_coords = coords_array[cluster_indices]
        diameter = self.calculate_cluster_diameter_fast(cluster_coords)
        
        if diameter <= self.max_distance_km:
            return False
            
        return True

    def geographic_split(self, cluster_indices, coords_array):
        """Enhanced splitting with DBSCAN for outlier handling"""
        n_points = len(cluster_indices)
        if n_points <= 2:
            return [cluster_indices]
            
        cluster_coords = coords_array[cluster_indices]
        
        # Handle outliers with DBSCAN for large clusters
        if n_points > 100:
            try:
                # Create distance matrix for DBSCAN
                distances = self.haversine_vectorized(cluster_coords)
                dbscan = DBSCAN(eps=self.max_distance_km/2, min_samples=3, 
                               metric='precomputed')
                labels = dbscan.fit_predict(distances)
                
                if len(np.unique(labels[labels != -1])) > 1:
                    return self._balance_split(cluster_indices, labels)
            except:
                pass  # Fall back to other methods
        
        # Small clusters: exact method
        if n_points <= 50:
            return self._exact_geographic_split(cluster_indices, cluster_coords)
            
        # Medium clusters: k-means with improved initialization
        if n_points <= 200:
            return self._kmeans_geographic_split(cluster_indices, cluster_coords)
            
        # Large clusters: hierarchical with complete linkage
        return self._hierarchical_geographic_split(cluster_indices, cluster_coords)

    def _exact_geographic_split(self, cluster_indices, cluster_coords):
        """Exact splitting for small clusters"""
        if len(cluster_coords) <= 2:
            return [cluster_indices]
        
        # Find the two points that are farthest apart
        max_distance = 0
        best_pair = (0, 1)
        
        for i in range(len(cluster_coords)):
            for j in range(i + 1, len(cluster_coords)):
                distance = self.haversine_single_pair(cluster_coords[i], cluster_coords[j])
                if distance > max_distance:
                    max_distance = distance
                    best_pair = (i, j)
        
        # Assign points to the closer of the two centers
        center1 = cluster_coords[best_pair[0]]
        center2 = cluster_coords[best_pair[1]]
        
        labels = np.zeros(len(cluster_coords))
        for i, coord in enumerate(cluster_coords):
            dist1 = self.haversine_single_pair(coord, center1)
            dist2 = self.haversine_single_pair(coord, center2)
            labels[i] = 0 if dist1 <= dist2 else 1
        
        return self._balance_split(cluster_indices, labels)

    def _kmeans_geographic_split(self, cluster_indices, cluster_coords):
        """K-means splitting for medium clusters"""
        try:
            # Use geographic coordinates directly for K-means
            kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
            labels = kmeans.fit_predict(cluster_coords)
            return self._balance_split(cluster_indices, labels)
        except:
            # Fallback to exact method
            return self._exact_geographic_split(cluster_indices, cluster_coords)

    def _hierarchical_geographic_split(self, cluster_indices, cluster_coords):
        """Improved hierarchical splitting with complete linkage"""
        n_sample = min(150, len(cluster_coords))
        sample_indices = np.random.choice(len(cluster_coords), n_sample, replace=False)
        sample_coords = cluster_coords[sample_indices]
        
        distances = self.haversine_pdist(sample_coords)
        linkage_matrix = linkage(distances, method='complete')
        sample_labels = fcluster(linkage_matrix, 2, criterion='maxclust') - 1
        
        # Assign based on nearest cluster center
        center1 = np.mean(sample_coords[sample_labels == 0], axis=0)
        center2 = np.mean(sample_coords[sample_labels == 1], axis=0)
        
        dist_to_center1 = self.haversine_vectorized(cluster_coords, center1.reshape(1, -1))[:, 0]
        dist_to_center2 = self.haversine_vectorized(cluster_coords, center2.reshape(1, -1))[:, 0]
        labels = (dist_to_center1 <= dist_to_center2).astype(int)
        
        return self._balance_split(cluster_indices, labels)

    def _balance_split(self, cluster_indices, labels):
        """Proximity-based balancing for spatial coherence"""
        unique_labels = np.unique(labels)
        if len(unique_labels) == 1:
            # All points have same label, split arbitrarily
            mid = len(cluster_indices) // 2
            return [cluster_indices[:mid], cluster_indices[mid:]]
        
        # Handle noise points from DBSCAN (label -1)
        if -1 in unique_labels:
            noise_mask = labels == -1
            valid_labels = labels[~noise_mask]
            if len(np.unique(valid_labels)) == 0:
                return [cluster_indices]
            
            # Assign noise points to nearest valid cluster
            for i in np.where(noise_mask)[0]:
                # Find nearest non-noise point
                distances = []
                for j in np.where(~noise_mask)[0]:
                    dist = self.haversine_single_pair(
                        self.full_coords_array[cluster_indices[i]], 
                        self.full_coords_array[cluster_indices[j]]
                    )
                    distances.append((dist, labels[j]))
                
                if distances:
                    labels[i] = min(distances, key=lambda x: x[0])[1]
        
        # Create clusters based on labels
        clusters = []
        for label in np.unique(labels):
            if label != -1:  # Skip noise label
                cluster_mask = labels == label
                cluster = cluster_indices[cluster_mask]
                if len(cluster) > 0:
                    clusters.append(cluster)
        
        if len(clusters) == 0:
            return [cluster_indices]
        elif len(clusters) == 1:
            # Split the single cluster arbitrarily
            cluster = clusters[0]
            mid = len(cluster) // 2
            return [cluster[:mid], cluster[mid:]]
        else:
            # Balance clusters if enabled
            if self.balance_clusters and len(clusters) == 2:
                return self._balance_two_clusters(clusters[0], clusters[1])
            return clusters

    def _balance_two_clusters(self, cluster_0, cluster_1):
        """Balance two clusters by size"""
        size0, size1 = len(cluster_0), len(cluster_1)
        
        if size0 <= 2 * size1 and size1 <= 2 * size0:
            return [cluster_0, cluster_1]
        
        coords0 = self.full_coords_array[cluster_0]
        coords1 = self.full_coords_array[cluster_1]
        
        # Balance only when size difference is significant
        if size0 > 2 * size1:
            center1 = np.mean(coords1, axis=0)
            dist_to_center1 = self.haversine_vectorized(coords0, center1.reshape(1, -1))[:, 0]
            n_move = min((size0 - size1) // 2, max(1, size1))
            move_idx = np.argsort(dist_to_center1)[:n_move]
            cluster_1 = np.concatenate([cluster_1, cluster_0[move_idx]])
            cluster_0 = np.delete(cluster_0, move_idx)
            
        elif size1 > 2 * size0:
            center0 = np.mean(coords0, axis=0)
            dist_to_center0 = self.haversine_vectorized(coords1, center0.reshape(1, -1))[:, 0]
            n_move = min((size1 - size0) // 2, max(1, size0))
            move_idx = np.argsort(dist_to_center0)[:n_move]
            cluster_0 = np.concatenate([cluster_0, cluster_1[move_idx]])
            cluster_1 = np.delete(cluster_1, move_idx)
            
        return [cluster_0, cluster_1]

    def divisive_clustering(self, customers_df):
        """Main clustering with cache management""" 
        customers_df = customers_df.copy().reset_index(drop=True)
        
        # Validate input
        if 'Latitude' not in customers_df.columns or 'Longitude' not in customers_df.columns:
            raise ValueError("DataFrame must contain 'Latitude' and 'Longitude' columns")
        
        coords_array = customers_df[['Latitude', 'Longitude']].values
        n_customers = len(customers_df)
        
        self.full_coords_array = coords_array
        self._distance_cache = {}  # Reset cache
        
        if n_customers == 0:
            customers_df['cluster'] = []
            return customers_df
        
        if n_customers == 1:
            customers_df['cluster'] = 1
            return customers_df
        
        # Priority queue approach for better clustering
        clusters_to_process = [(n_customers, np.arange(n_customers))]  # (size, indices)
        final_clusters = []
        
        iteration_count = 0
        max_iterations = n_customers * 2
        
        while clusters_to_process and iteration_count < max_iterations:
            # Process largest cluster first
            clusters_to_process.sort(key=lambda x: x[0], reverse=True)
            current_size, current_cluster_indices = clusters_to_process.pop(0)
            iteration_count += 1
            
            if self.should_split_cluster(current_cluster_indices, coords_array):
                subclusters = self.geographic_split(current_cluster_indices, coords_array)
                
                for subcluster_indices in subclusters:
                    if len(subcluster_indices) > 0:
                        clusters_to_process.append((len(subcluster_indices), subcluster_indices))
            else:
                final_clusters.append(current_cluster_indices)
        
        # Handle remaining clusters
        final_clusters.extend([indices for _, indices in clusters_to_process])
        
        # Create result DataFrame
        result_df = customers_df.copy()
        result_df['cluster'] = -1
        
        for cluster_id, cluster_indices in enumerate(final_clusters, 1):
            result_df.loc[cluster_indices, 'cluster'] = cluster_id
        
        # Clean up stored data
        if hasattr(self, 'full_coords_array'):
            del self.full_coords_array
        self._distance_cache = {}

        return result_df
    
    def get_cluster_stats(self, clustered_df):
        """Get comprehensive clustering statistics."""
        if 'cluster' not in clustered_df.columns:
            raise ValueError("DataFrame must contain 'cluster' column")
        
        stats = {}
        cluster_sizes = []
        cluster_diameters = []
        
        for cluster_id in clustered_df['cluster'].unique():
            if cluster_id == -1:
                continue
                
            cluster_data = clustered_df[clustered_df['cluster'] == cluster_id]
            coords = cluster_data[['Latitude', 'Longitude']].values
            
            diameter = self.calculate_cluster_diameter_fast(coords)
            cluster_sizes.append(len(cluster_data))
            cluster_diameters.append(diameter)
            
            stats[cluster_id] = {
                'size': len(cluster_data),
                'diameter_km': diameter,
                'centroid_lat': np.mean(coords[:, 0]),
                'centroid_lon': np.mean(coords[:, 1]),
                'meets_size_constraint': len(cluster_data) <= self.max_customers_per_cluster,
                'meets_distance_constraint': diameter <= self.max_distance_km
            }
        
        # Overall statistics
        if cluster_sizes:  # Only calculate if there are clusters
            stats['summary'] = {
                'total_clusters': len(stats) - 1,  # Excluding summary key
                'avg_cluster_size': np.mean(cluster_sizes),
                'max_cluster_size': np.max(cluster_sizes),
                'min_cluster_size': np.min(cluster_sizes),
                'avg_diameter': np.mean(cluster_diameters),
                'max_diameter': np.max(cluster_diameters),
                'size_violations': sum(1 for size in cluster_sizes if size > self.max_customers_per_cluster),
                'distance_violations': sum(1 for diameter in cluster_diameters if diameter > self.max_distance_km)
            }
        else:
            stats['summary'] = {
                'total_clusters': 0,
                'avg_cluster_size': 0,
                'max_cluster_size': 0,
                'min_cluster_size': 0,
                'avg_diameter': 0,
                'max_diameter': 0,
                'size_violations': 0,
                'distance_violations': 0
            }
        
        return stats

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numba

### Fastest
class OptimizedDivisiveGeographicClustering_fast:
    def __init__(self, max_customers_per_cluster=20, max_distance_km=50):
        self.max_customers_per_cluster = max_customers_per_cluster
        self.max_distance_km = max_distance_km
        self.earth_radius_km = 6371.0

    @staticmethod
    @numba.njit(fastmath=True)
    def haversine_single_pair(lat1, lon1, lat2, lon2):
        """Numba-optimized haversine distance between two points."""
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
        return 6371.0 * 2 * np.arcsin(np.sqrt(min(1.0, a)))
    
    @staticmethod
    @numba.njit(fastmath=True, parallel=True)
    def haversine_vectorized(coord, coords):
        """Vectorized haversine distance from one point to many."""
        lat1, lon1 = np.radians(coord)
        dists = np.empty(coords.shape[0])
        for i in numba.prange(coords.shape[0]):
            lat2, lon2 = np.radians(coords[i])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
            dists[i] = 6371.0 * 2 * np.arcsin(np.sqrt(min(1.0, a)))
        return dists

    def compute_diameter_and_farthest_pair(self, coords):
        """Compute diameter and farthest pair with optimal strategy."""
        n = len(coords)
        if n <= 100:  # Exact for small clusters
            dists = self.haversine_pdist(coords)
            dist_matrix = squareform(dists)
            max_idx = np.unravel_index(np.argmax(dist_matrix), dist_matrix.shape)
            return dist_matrix[max_idx], max_idx
        else:  # Approximate for large clusters
            return self._two_pass_approximation(coords)

    def _two_pass_approximation(self, coords):
        """Two-pass algorithm for approximate diameter and farthest pair."""
        # First pass: random point to farthest point
        idx0 = np.random.randint(len(coords))
        dists = self.haversine_vectorized(coords[idx0], coords)
        idx1 = np.argmax(dists)
        
        # Second pass: farthest point to its farthest point
        dists = self.haversine_vectorized(coords[idx1], coords)
        idx2 = np.argmax(dists)
        diameter = dists[idx2]
        
        return diameter, (idx1, idx2)

    def haversine_pdist(self, coords):
        """Optimized haversine pairwise distances using vectorization."""
        n = coords.shape[0]
        dists = np.zeros(n*(n-1)//2)
        k = 0
        for i in range(n):
            dists_i = self.haversine_vectorized(coords[i], coords[i+1:])
            dists[k:k+len(dists_i)] = dists_i
            k += len(dists_i)
        return dists

    def should_split_cluster(self, cluster_indices, coords_array):
        """Determine if cluster should be split with early termination."""
        cluster_size = len(cluster_indices)
        
        if cluster_size <= 1:
            return False, None
            
        if cluster_size > self.max_customers_per_cluster:
            return True, None  # Size violation
        
        # Check diameter constraint
        cluster_coords = coords_array[cluster_indices]
        diameter, farthest_pair = self.compute_diameter_and_farthest_pair(cluster_coords)
        return diameter > self.max_distance_km, farthest_pair

    def geographic_split(self, cluster_indices, coords_array, farthest_pair=None):
        """Efficient cluster splitting with optional precomputed centers."""
        cluster_coords = coords_array[cluster_indices]
        n = len(cluster_coords)
        
        # Get or compute farthest pair
        if farthest_pair is None:
            if n <= 100:
                _, farthest_pair = self.compute_diameter_and_farthest_pair(cluster_coords)
            else:
                _, farthest_pair = self._two_pass_approximation(cluster_coords)
        
        center1_idx, center2_idx = farthest_pair
        center1 = cluster_coords[center1_idx]
        center2 = cluster_coords[center2_idx]
        
        # Vectorized distance calculations
        dists1 = self.haversine_vectorized(center1, cluster_coords)
        dists2 = self.haversine_vectorized(center2, cluster_coords)
        labels = (dists1 <= dists2).astype(int)
        
        # Create subclusters
        mask = labels.astype(bool)
        cluster_a = cluster_indices[mask]
        cluster_b = cluster_indices[~mask]
        
        # Balance clusters if needed
        if len(cluster_a) == 0 or len(cluster_b) == 0:
            return self._split_fallback(cluster_indices, cluster_coords)
            
        return [cluster_a, cluster_b]

    def _split_fallback(self, cluster_indices, cluster_coords):
        """Fallback split when primary method fails."""
        # Use longitude-based split as fallback
        sorted_idx = np.argsort(cluster_coords[:, 1])
        mid = len(sorted_idx) // 2
        return [
            cluster_indices[sorted_idx[:mid]],
            cluster_indices[sorted_idx[mid:]]
        ]

    def divisive_clustering(self, customers_df):
        """Optimized divisive clustering with efficient diameter checks."""
        # Initialization and validation
        if 'Latitude' not in customers_df or 'Longitude' not in customers_df:
            raise ValueError("Missing Latitude/Longitude columns")
            
        coords_array = customers_df[['Latitude', 'Longitude']].values
        n = len(customers_df)
        
        # Edge cases
        if n == 0:
            return customers_df.assign(cluster=pd.Series(dtype=int))
        if n == 1:
            return customers_df.assign(cluster=1)
        
        # Initialize clustering
        clusters_to_process = [np.arange(n)]
        final_clusters = []
        
        # Process clusters
        while clusters_to_process:
            current = clusters_to_process.pop(0)
            should_split, farthest_pair = self.should_split_cluster(current, coords_array)
            
            if should_split:
                subclusters = self.geographic_split(
                    current, coords_array, farthest_pair
                )
                clusters_to_process.extend(subclusters)
            else:
                final_clusters.append(current)
        
        # Assign cluster labels
        cluster_labels = np.zeros(n, dtype=int)
        for idx, cluster in enumerate(final_clusters, 1):
            cluster_labels[cluster] = idx
            
        return customers_df.assign(cluster=cluster_labels)

    def get_cluster_stats(self, clustered_df):
        """Get comprehensive clustering statistics."""
        if 'cluster' not in clustered_df.columns:
            raise ValueError("DataFrame must contain 'cluster' column")
        
        stats = {}
        cluster_sizes = []
        cluster_diameters = []
        
        for cluster_id in clustered_df['cluster'].unique():
            if cluster_id == -1:
                continue
                
            cluster_data = clustered_df[clustered_df['cluster'] == cluster_id]
            coords = cluster_data[['Latitude', 'Longitude']].values
            n_points = len(coords)
            
            # Handle diameter calculation efficiently
            if n_points == 0:
                diameter = 0.0
            elif n_points == 1:
                diameter = 0.0
            else:
                diameter, _ = self.compute_diameter_and_farthest_pair(coords)
            
            cluster_sizes.append(n_points)
            cluster_diameters.append(diameter)
            
            stats[cluster_id] = {
                'size': n_points,
                'diameter_km': diameter,
                'centroid_lat': np.mean(coords[:, 0]) if n_points > 0 else None,
                'centroid_lon': np.mean(coords[:, 1]) if n_points > 0 else None,
                'meets_size_constraint': n_points <= self.max_customers_per_cluster,
                'meets_distance_constraint': diameter <= self.max_distance_km
            }
        
        # Calculate overall statistics
        total_clusters = len(stats)
        size_violations = sum(1 for size in cluster_sizes if size > self.max_customers_per_cluster)
        distance_violations = sum(1 for d in cluster_diameters if d > self.max_distance_km)
        
        # Handle empty case
        summary = {
            'total_clusters': total_clusters,
            'size_violations': size_violations,
            'distance_violations': distance_violations,
        }
        
        # Add statistical measures only if clusters exist
        if cluster_sizes:
            summary.update({
                'avg_cluster_size': np.mean(cluster_sizes),
                'max_cluster_size': np.max(cluster_sizes),
                'min_cluster_size': np.min(cluster_sizes),
                'avg_diameter': np.mean(cluster_diameters),
                'max_diameter': np.max(cluster_diameters),
            })
        else:
            summary.update({
                'avg_cluster_size': 0,
                'max_cluster_size': 0,
                'min_cluster_size': 0,
                'avg_diameter': 0,
                'max_diameter': 0,
            })
        
        stats['summary'] = summary
        return stats

In [ ]:
# =============================================================================
# 2. DIVISIVE HIERARCHICAL CLUSTERING
# =============================================================================

from clustering.divisive_clustering import DivisiveGeographicClustering, OptimizedDivisiveGeographicClustering

print("\n" + "="*60)
print("2. DIVISIVE HIERARCHICAL CLUSTERING")
print("="*60)

# divisive_clusterer = OptimizedDivisiveGeographicClustering( # Rivers: Too Long --
#     max_customers_per_cluster=20,  # REQUIRED
#     max_distance_km=5            # REQUIRED  
# ) 

divisive_clusterer = OptimizedDivisiveGeographicClustering_b( 
    # Rivers: Divisive clusters created: 48 || Silhouette Score: 0.54 || Constraint violations: Size=6, Distance=3
    max_customers_per_cluster=20,  # REQUIRED
    max_distance_km=5            # REQUIRED
    ,use_vectorized_distances=True, balance_clusters=False
)

# divisive_clusterer = OptimizedDivisiveGeographicClustering_be( 
#     # Rivers: Divisive clusters created: 26 || Silhouette Score: 0.57 || Constraint violations: Size=9, Distance=6
#     max_customers_per_cluster=20,  # REQUIRED
#     max_distance_km=10            # REQUIRED
#     ,use_vectorized_distances=True, balance_clusters=False
# )

# divisive_clusterer = OptimizedDivisiveGeographicClustering_bo( 
# # Rivers: Divisive clusters created: 60 || Silhouette Score: 0.58 || Constraint violations: Size=9, Distance=7
#     max_customers_per_cluster=20,  # REQUIRED
#     max_distance_km=5            # REQUIRED 
# ) 

# divisive_clusterer = OptimizedDivisiveGeographicClustering_fast(
#     # Rivers: Divisive clusters created: 63 || Silhouette Score: 0.46 || Constraint violations: Size=0, Distance=0
#     max_customers_per_cluster=20,  # REQUIRED
#     max_distance_km=5            # REQUIRED
# )

# divisive_clusterer = OptimizedDivisiveGeographicClustering_Main( 
#     # Rivers: Divisive clusters created: 69 ||Silhouette Score: 0.41
#     max_customers_per_cluster=20,  # REQUIRED
#     max_distance_km=5            # REQUIRED 
# ) 

divisive_result = divisive_clusterer.divisive_clustering(df_all_cluster.copy())
print(f"\nDivisive clusters created: {divisive_result['cluster'].nunique()}")
# print(f"Cluster sizes: {divisive_result['cluster'].value_counts().sort_index().head()}")
_ = evaluate_unsupervised_clustering(divisive_result)

# # Get detailed statistics
stats = divisive_clusterer.get_cluster_stats(divisive_result)
print(f"Total clusters: {stats['summary']['total_clusters']}")
print(f"Constraint violations: Size={stats['summary']['size_violations']}, Distance={stats['summary']['distance_violations']}")
 

In [ ]:
divisive_result.cluster.value_counts().reset_index().head(10)

In [ ]:
vis_and_save(df_routes = divisive_result,
                 df_stockpoint = None,   
                 filename=None,
                 cluster_col = 'cluster')

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

class BaseGeographicClustering:
    """
    A base class containing common geographic utility methods
    used by both Divisive and Agglomerative clustering implementations.
    """
    def __init__(self):
        self.earth_radius_km = 6371.0

    def haversine_vectorized(self, coords1, coords2=None):
        """
        Highly optimized vectorized haversine distance calculation.
        If coords2 is None, calculates pairwise distances within coords1.
        Assumes coords are [latitude, longitude].
        """
        if coords2 is None:
            # Pairwise distances within coords1 (NxN matrix)
            coords1_rad = np.radians(coords1)
            
            lat1 = coords1_rad[:, 0]
            lon1 = coords1_rad[:, 1]
            
            lat1_mesh, lat2_mesh = np.meshgrid(lat1, lat1)
            lon1_mesh, lon2_mesh = np.meshgrid(lon1, lon1)
            
            dlat = lat2_mesh - lat1_mesh
            dlon = lon2_mesh - lon1_mesh
            
            a = (np.sin(dlat / 2) ** 2 + 
                 np.cos(lat1_mesh) * np.cos(lat2_mesh) * np.sin(dlon / 2) ** 2)
            c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
            
            return self.earth_radius_km * c
        else:
            # Distance from each point in coords1 to each point in coords2 (NxM matrix)
            coords1_rad = np.radians(coords1)
            coords2_rad = np.radians(coords2)
            
            lat1 = coords1_rad[:, 0][:, np.newaxis]
            lon1 = coords1_rad[:, 1][:, np.newaxis]
            lat2 = coords2_rad[:, 0][np.newaxis, :]
            lon2 = coords2_rad[:, 1][np.newaxis, :]
            
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            
            a = (np.sin(dlat / 2) ** 2 + 
                 np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
            c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
            
            return self.earth_radius_km * c
    
    def haversine_pdist(self, coords):
        """
        Optimized haversine distance calculation for pdist, returning condensed distance matrix.
        Assumes coords are [latitude, longitude].
        """
        def haversine_metric(u, v):
            lat1, lon1 = np.radians(u)
            lat2, lon2 = np.radians(v)
            
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            
            a = (np.sin(dlat / 2) ** 2 + 
                 np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
            c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
            
            return self.earth_radius_km * c
        
        return pdist(coords, metric=haversine_metric)
    
    def haversine_single_pair(self, coord1, coord2):
        """Calculate haversine distance between two points."""
        lat1, lon1 = np.radians(coord1)
        lat2, lon2 = np.radians(coord2)
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = (np.sin(dlat / 2) ** 2 + 
             np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2)
        c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
        
        return self.earth_radius_km * c

    def _convex_hull_diameter(self, coords):
        """Enhanced convex hull approximation for diameter estimation."""
        lats, lons = coords[:, 0], coords[:, 1]
        
        # Get extreme points (min/max lat/lon)
        extreme_indices = [
            np.argmax(lats), np.argmin(lats),
            np.argmax(lons), np.argmin(lons)
        ]
        
        # Add points from different quadrants relative to the mean center
        lat_center, lon_center = np.mean(lats), np.mean(lons)
        
        quadrants = [
            (lats >= lat_center) & (lons >= lon_center),   # NE
            (lats >= lat_center) & (lons < lon_center),    # NW
            (lats < lat_center) & (lons >= lon_center),    # SE
            (lats < lat_center) & (lons < lon_center)      # SW
        ]
        
        for quadrant in quadrants:
            if np.any(quadrant):
                quad_indices = np.where(quadrant)[0]
                # Add furthest point from center in each non-empty quadrant
                distances_from_center = np.sqrt(
                    (lats[quad_indices] - lat_center)**2 + 
                    (lons[quad_indices] - lon_center)**2
                )
                furthest_idx = quad_indices[np.argmax(distances_from_center)]
                extreme_indices.append(furthest_idx)
        
        # Add some random points to further improve approximation for larger clusters
        n_random = min(12, len(coords) - len(set(extreme_indices)))
        if n_random > 0:
            available_indices = list(set(range(len(coords))) - set(extreme_indices))
            if available_indices:
                random_indices = np.random.choice(available_indices, 
                                                  min(n_random, len(available_indices)), 
                                                  replace=False)
                extreme_indices.extend(random_indices)
        
        # Get unique sample coordinates
        sample_indices = list(set(extreme_indices))
        sample_coords = coords[sample_indices]
        
        if len(sample_coords) <= 1:
            return 0
        
        # Calculate max distance among the sampled points
        distances = self.haversine_pdist(sample_coords)
        return np.max(distances)
    
    def _grid_based_diameter(self, coords):
        """Grid-based sampling for very large clusters."""
        lats, lons = coords[:, 0], coords[:, 1]
        
        # Create 6x6 grid
        lat_bins = np.linspace(lats.min(), lats.max(), 7)
        lon_bins = np.linspace(lons.min(), lons.max(), 7)
        
        sample_indices = []
        for i in range(len(lat_bins)-1):
            for j in range(len(lon_bins)-1):
                mask = ((lats >= lat_bins[i]) & (lats < lat_bins[i+1]) & 
                        (lons >= lon_bins[j]) & (lons < lon_bins[j+1]))
                cell_indices = np.where(mask)[0]
                if len(cell_indices) > 0:
                    # Sample up to 2 points from each cell
                    n_sample = min(2, len(cell_indices))
                    sampled = np.random.choice(cell_indices, n_sample, replace=False)
                    sample_indices.extend(sampled)
        
        if len(sample_indices) <= 1:
            return 0
        
        sample_coords = coords[sample_indices]
        distances = self.haversine_pdist(sample_coords)
        return np.max(distances)
    
    def calculate_cluster_diameter_fast(self, coords):
        """
        Fast cluster diameter calculation with multiple optimization strategies.
        """
        n_points = len(coords)
        
        if n_points <= 1:
            return 0
        
        if n_points == 2:
            return self.haversine_single_pair(coords[0], coords[1])
        
        # Use different strategies based on cluster size
        if n_points <= 10:
            # Small clusters: exact calculation using pdist
            distances = self.haversine_pdist(coords)
            return np.max(distances)
        elif n_points <= 50:
            # Medium clusters: vectorized calculation or pdist
            # Prioritize vectorized if available and faster for this range
            distance_matrix = self.haversine_vectorized(coords)
            # Ensure we're not taking max of diagonal (self-distances = 0)
            return np.max(distance_matrix[np.triu_indices(n_points, k=1)])
        else:
            # Large clusters: smart sampling
            return self._smart_diameter_estimation(coords)
    
    def _smart_diameter_estimation(self, coords):
        """
        Improved diameter estimation using multiple sampling strategies.
        """
        n_points = len(coords)
        
        # Strategy 1: Convex hull approximation
        hull_diameter = self._convex_hull_diameter(coords)
        
        # Strategy 2: Grid-based sampling for very large clusters (higher confidence for max)
        if n_points > 200: # Threshold for when grid sampling might be beneficial
            grid_diameter = self._grid_based_diameter(coords)
            return max(hull_diameter, grid_diameter)
        
        return hull_diameter

    def _find_approximate_farthest_pair(self, coords):
        """Find approximate farthest pair for large clusters.
           Moved from OptimizedDivisiveGeographicClustering to BaseGeographicClustering."""
        n_points = len(coords)
        
        if n_points <= 100:
            # For moderate sizes, use exact calculation
            distances = self.haversine_pdist(coords)
            distance_matrix = squareform(distances)
            max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
            return max_idx[0], max_idx[1]
        
        # For large clusters, use sampling
        sample_size = min(50, n_points)
        sample_indices = np.random.choice(n_points, sample_size, replace=False)
        sample_coords = coords[sample_indices]
        
        distances = self.haversine_pdist(sample_coords)
        distance_matrix = squareform(distances)
        max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
        
        return sample_indices[max_idx[0]], sample_indices[max_idx[1]]

    def get_cluster_stats(self, clustered_df, max_customers_per_cluster, max_distance_km):
        """Get comprehensive clustering statistics."""
        if 'cluster' not in clustered_df.columns:
            raise ValueError("DataFrame must contain 'cluster' column")
        
        stats = {}
        cluster_sizes = []
        cluster_diameters = []
        
        for cluster_id in clustered_df['cluster'].unique():
            if cluster_id == -1: # Unassigned points if any
                continue
            
            cluster_data = clustered_df[clustered_df['cluster'] == cluster_id]
            coords = cluster_data[['Latitude', 'Longitude']].values
            
            diameter = self.calculate_cluster_diameter_fast(coords)
            cluster_sizes.append(len(cluster_data))
            cluster_diameters.append(diameter)
            
            stats[cluster_id] = {
                'size': len(cluster_data),
                'diameter_km': diameter,
                'centroid_lat': np.mean(coords[:, 0]),
                'centroid_lon': np.mean(coords[:, 1]),
                'meets_size_constraint': len(cluster_data) <= max_customers_per_cluster,
                'meets_distance_constraint': diameter <= max_distance_km
            }
        
        # Overall statistics
        stats['summary'] = {
            'total_clusters': len(stats), 
            'avg_cluster_size': np.mean(cluster_sizes) if cluster_sizes else 0,
            'max_cluster_size': np.max(cluster_sizes) if cluster_sizes else 0,
            'min_cluster_size': np.min(cluster_sizes) if cluster_sizes else 0,
            'avg_diameter': np.mean(cluster_diameters) if cluster_diameters else 0,
            'max_diameter': np.max(cluster_diameters) if cluster_diameters else 0,
            'size_violations': sum(1 for size in cluster_sizes if size > max_customers_per_cluster),
            'distance_violations': sum(1 for diameter in cluster_diameters if diameter > max_distance_km)
        }
        
        return stats


class AgglomerativeGeographicClustering(BaseGeographicClustering):
    def __init__(self, max_customers_per_cluster=20, max_distance_km=50, 
                 linkage_method='ward', sub_cluster_if_oversized=True):
        """
        Initializes the Agglomerative Geographic Clustering.

        Args:
            max_customers_per_cluster (int): Maximum number of customers allowed in a single cluster.
            max_distance_km (float): Maximum diameter (distance between two farthest points)
                                     allowed within a cluster in kilometers.
            linkage_method (str): Method to use for calculating the distance between clusters
                                  in hierarchical clustering. Options: 'ward', 'single', 'complete', 'average'.
            sub_cluster_if_oversized (bool): If True, clusters that exceed max_customers_per_cluster
                                             after distance-based cutting will be further sub-clustered
                                             using K-Means.
        """
        super().__init__()
        self.max_customers_per_cluster = max_customers_per_cluster
        self.max_distance_km = max_distance_km
        self.linkage_method = linkage_method
        self.sub_cluster_if_oversized = sub_cluster_if_oversized

    def agglomerative_clustering(self, customers_df):
        """
        Performs agglomerative hierarchical clustering on geographic data
        with constraints on cluster size and diameter.
        """
        customers_df = customers_df.copy().reset_index(drop=True)
        
        if 'Latitude' not in customers_df.columns or 'Longitude' not in customers_df.columns:
            raise ValueError("DataFrame must contain 'Latitude' and 'Longitude' columns")
        
        coords_array = customers_df[['Latitude', 'Longitude']].values
        n_customers = len(customers_df)
        
        if n_customers == 0:
            customers_df['cluster'] = []
            return customers_df
        
        if n_customers == 1:
            customers_df['cluster'] = 1
            return customers_df

        # Step 1: Calculate pairwise Haversine distances
        print(f"Calculating {n_customers*(n_customers-1)//2} pairwise distances...")
        # Check if the number of points is too large for pdist to avoid MemoryError
        # A rough heuristic: 5000 points * 5000 points / 2 * 8 bytes/float ~ 100MB
        # For very large N, consider approximate methods if pdist is too slow/memory intensive
        if n_customers > 2000 and self.linkage_method != 'ward': # Ward only works with Euclidean-like pdist
             # For very large datasets, pdist might be too slow or memory intensive.
             # In such cases, one might consider sampling or approximate hierarchical methods,
             # or other clustering algorithms like DBSCAN that don't require a full distance matrix.
             # For now, we proceed with pdist as it's standard for scipy.hierarchy.
            print("Warning: Large dataset for pdist. This might take a while or consume a lot of memory.")

        distances = self.haversine_pdist(coords_array)
        
        # Step 2: Perform hierarchical clustering using linkage
        print(f"Performing linkage using '{self.linkage_method}' method...")
        linkage_matrix = linkage(distances, method=self.linkage_method)
        
        # Step 3: Cut the dendrogram based on max_distance_km
        # This creates clusters where no two points are farther apart than max_distance_km
        print(f"Cutting dendrogram at max_distance_km={self.max_distance_km}...")
        initial_labels = fcluster(linkage_matrix, self.max_distance_km, criterion='distance')
        
        customers_df['cluster_temp'] = initial_labels
        final_cluster_id = 0
        final_clusters = {}

        # Step 4: Post-process for max_customers_per_cluster constraint
        print(f"Post-processing clusters for size constraint (max {self.max_customers_per_cluster} customers)...")
        for current_cluster_label in sorted(customers_df['cluster_temp'].unique()):
            cluster_indices = customers_df[customers_df['cluster_temp'] == current_cluster_label].index.values
            current_coords = coords_array[cluster_indices]
            
            if len(cluster_indices) > self.max_customers_per_cluster and self.sub_cluster_if_oversized:
                print(f"  Cluster {current_cluster_label} (size {len(cluster_indices)}) is oversized. Sub-clustering...")
                # Sub-cluster using K-Means. Determine optimal k based on current size / max_customers_per_cluster
                k_sub = int(np.ceil(len(cluster_indices) / self.max_customers_per_cluster))
                k_sub = max(2, k_sub) # Ensure at least 2 clusters if splitting
                
                # Use approximate farthest pair for K-Means initialization
                initial_centers_indices = self._find_approximate_farthest_pair(current_coords)
                # Ensure initial_centers_indices has enough elements for k_sub.
                # If k_sub > 2, KMeans++ initialization is generally more robust than a simple farthest pair.
                if k_sub > 2:
                    kmeans_sub = KMeans(n_clusters=k_sub, n_init=10, random_state=42) # Let KMeans find its own init
                elif len(initial_centers_indices) >= 2: # k_sub = 2, use the farthest pair if available
                    kmeans_sub = KMeans(n_clusters=k_sub, init=current_coords[[initial_centers_indices[0], initial_centers_indices[1]]], n_init=1, random_state=42)
                else: # Fallback if initial_centers_indices is not sufficient for k_sub=2
                    kmeans_sub = KMeans(n_clusters=k_sub, n_init=10, random_state=42)


                sub_labels = kmeans_sub.fit_predict(current_coords)
                
                for sub_label in np.unique(sub_labels):
                    final_cluster_id += 1
                    sub_cluster_indices = cluster_indices[sub_labels == sub_label]
                    final_clusters[final_cluster_id] = sub_cluster_indices
            else:
                final_cluster_id += 1
                final_clusters[final_cluster_id] = cluster_indices
        
        # Assign final cluster labels to the DataFrame
        result_df = customers_df.copy()
        result_df['cluster'] = -1 # Initialize with unassigned

        for cluster_id, indices in final_clusters.items():
            result_df.loc[indices, 'cluster'] = cluster_id
        
        result_df = result_df.drop(columns=['cluster_temp'])
        print("Agglomerative clustering completed.\n")
        return result_df

class OptimizedDivisiveGeographicClustering(BaseGeographicClustering):
    """ Best - Fast and Accurate Divisive Geographic Clustering
    """
    def __init__(self, max_customers_per_cluster=20, max_distance_km=50, 
                 use_vectorized_distances=True, balance_clusters=False):
        super().__init__()
        self.max_customers_per_cluster = max_customers_per_cluster
        self.max_distance_km = max_distance_km
        self.use_vectorized_distances = use_vectorized_distances
        self.balance_clusters = balance_clusters
        
    def calculate_cluster_diameter_fast(self, coords): # Overrides Base class method
        """
        Fast cluster diameter calculation with multiple optimization strategies.
        """
        n_points = len(coords)
        
        if n_points <= 1:
            return 0
        
        if n_points == 2:
            return self.haversine_single_pair(coords[0], coords[1])
        
        # Use different strategies based on cluster size
        if n_points <= 10:
            # Small clusters: exact calculation
            distances = self.haversine_pdist(coords)
            return np.max(distances)
        elif n_points <= 50:
            # Medium clusters: vectorized calculation
            if self.use_vectorized_distances:
                distance_matrix = self.haversine_vectorized(coords)
                return np.max(distance_matrix)
            else:
                distances = self.haversine_pdist(coords)
                return np.max(distances)
        else:
            # Large clusters: smart sampling
            return self._smart_diameter_estimation(coords)
    
    def _smart_diameter_estimation(self, coords): # Overrides Base class method
        """
        Improved diameter estimation using multiple sampling strategies.
        """
        n_points = len(coords)
        
        # Strategy 1: Convex hull approximation
        hull_diameter = self._convex_hull_diameter(coords)
        
        # Strategy 2: Grid-based sampling for large clusters
        if n_points > 200:
            grid_diameter = self._grid_based_diameter(coords)
            return max(hull_diameter, grid_diameter)
        
        return hull_diameter
    
    def should_split_cluster(self, cluster_indices, coords_array):
        """Enhanced cluster splitting logic with load balancing."""
        cluster_size = len(cluster_indices)
        
        if cluster_size <= 2:
            return False
        
        # Hard size constraint
        if cluster_size > self.max_customers_per_cluster * 1.5:
            return True
        
        # Soft size constraint with diameter check
        if cluster_size > self.max_customers_per_cluster:
            cluster_coords = coords_array[cluster_indices]
            diameter = self.calculate_cluster_diameter_fast(cluster_coords)
            return diameter > self.max_distance_km * 0.8  # More lenient for size
        
        # Diameter constraint
        cluster_coords = coords_array[cluster_indices]
        diameter = self.calculate_cluster_diameter_fast(cluster_coords)
        
        return diameter > self.max_distance_km
    
    def geographic_split(self, cluster_indices, coords_array):
        """
        Improved geographic splitting with better load balancing.
        """
        if len(cluster_indices) <= 2:
            return [cluster_indices]
        
        cluster_coords = coords_array[cluster_indices]
        n_points = len(cluster_coords)
        
        # For small clusters, use exact method
        if n_points <= 50:
            return self._exact_geographic_split(cluster_indices, cluster_coords)
        
        # For medium clusters, use K-means with geographic initialization
        if n_points <= 200:
            return self._kmeans_geographic_split(cluster_indices, cluster_coords)
        
        # For large clusters, use hierarchical approach
        return self._hierarchical_geographic_split(cluster_indices, cluster_coords)
    
    def _exact_geographic_split(self, cluster_indices, cluster_coords):
        """Exact splitting for small clusters."""
        n_points = len(cluster_coords)
        
        # Find the two points that are farthest apart
        distances = self.haversine_pdist(cluster_coords)
        distance_matrix = squareform(distances)
        max_idx = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
        center1_idx, center2_idx = max_idx[0], max_idx[1]
        
        center1 = cluster_coords[center1_idx]
        center2 = cluster_coords[center2_idx]
        
        # Assign points to closest center
        distances_to_center1 = self.haversine_vectorized(cluster_coords, center1.reshape(1, -1))[:, 0]
        distances_to_center2 = self.haversine_vectorized(cluster_coords, center2.reshape(1, -1))[:, 0]
        
        labels = (distances_to_center1 <= distances_to_center2).astype(int)
        
        return self._balance_split(cluster_indices, labels)
    
    def _kmeans_geographic_split(self, cluster_indices, cluster_coords):
        """K-means splitting with geographic initialization."""
        n_points = len(cluster_coords)
        
        # Initialize with farthest pair
        # This calls _find_approximate_farthest_pair from BaseGeographicClustering
        center1_idx, center2_idx = self._find_approximate_farthest_pair(cluster_coords) 
        initial_centers = cluster_coords[[center1_idx, center2_idx]]
        
        # Apply K-means
        kmeans = KMeans(n_clusters=2, init=initial_centers, n_init=1, random_state=42)
        labels = kmeans.fit_predict(cluster_coords)
        
        return self._balance_split(cluster_indices, labels)
    
    def _hierarchical_geographic_split(self, cluster_indices, cluster_coords):
        """Hierarchical splitting for large clusters."""
        # Use linkage-based clustering for very large clusters
        n_sample = min(100, len(cluster_coords))
        sample_indices = np.random.choice(len(cluster_coords), n_sample, replace=False)
        sample_coords = cluster_coords[sample_indices]
        
        # Compute linkage on sample
        distances = self.haversine_pdist(sample_coords)
        linkage_matrix = linkage(distances, method='ward')
        sample_labels = fcluster(linkage_matrix, 2, criterion='maxclust') - 1
        
        # Ensure that both sub-clusters have points from the sample
        center1_coords = sample_coords[sample_labels == 0]
        center2_coords = sample_coords[sample_labels == 1]
        
        if len(center1_coords) == 0 or len(center2_coords) == 0:
            # Fallback to exact split if hierarchical sample leads to empty sub-clusters
            return self._exact_geographic_split(cluster_indices, cluster_coords)
        
        center1 = np.mean(center1_coords, axis=0)
        center2 = np.mean(center2_coords, axis=0)
        
        distances_to_center1 = self.haversine_vectorized(cluster_coords, center1.reshape(1, -1))[:, 0]
        distances_to_center2 = self.haversine_vectorized(cluster_coords, center2.reshape(1, -1))[:, 0]
        
        labels = (distances_to_center1 <= distances_to_center2).astype(int)
        
        return self._balance_split(cluster_indices, labels)
    
    def _balance_split(self, cluster_indices, labels):
        """Balance the split to avoid very uneven clusters."""
        cluster_0_indices = cluster_indices[labels == 0]
        cluster_1_indices = cluster_indices[labels == 1]
        
        # Ensure no empty clusters (very important for recursive calls)
        if len(cluster_0_indices) == 0:
            # If one cluster is empty, move one point from the other to it.
            # This handles edge cases but might lead to a single point cluster,
            # which the should_split_cluster check will prevent further splitting if <= 2.
            if len(cluster_1_indices) > 0:
                cluster_0_indices = np.array([cluster_1_indices[0]])
                cluster_1_indices = cluster_1_indices[1:]
            else: # Both empty, should not happen if initial cluster_indices was not empty
                return [np.array([]), np.array([])]
        elif len(cluster_1_indices) == 0:
            if len(cluster_0_indices) > 0:
                cluster_1_indices = np.array([cluster_0_indices[0]])
                cluster_0_indices = cluster_0_indices[1:]
            else: # Both empty
                return [np.array([]), np.array([])]
        
        # Optional: Balance cluster sizes if one is much larger
        if self.balance_clusters:
            size_0, size_1 = len(cluster_0_indices), len(cluster_1_indices)
            if size_0 > 3 * size_1 and size_1 > 0: # If cluster 0 is significantly larger
                n_move = (size_0 - size_1) // 4 # Move a quarter of the difference
                move_indices = cluster_0_indices[:n_move]
                cluster_0_indices = cluster_0_indices[n_move:]
                cluster_1_indices = np.concatenate([cluster_1_indices, move_indices])
            elif size_1 > 3 * size_0 and size_0 > 0: # If cluster 1 is significantly larger
                n_move = (size_1 - size_0) // 4
                move_indices = cluster_1_indices[:n_move]
                cluster_1_indices = cluster_1_indices[n_move:]
                cluster_0_indices = np.concatenate([cluster_0_indices, move_indices])
        
        return [cluster_0_indices, cluster_1_indices]
    
    def divisive_clustering(self, customers_df):
        """Perform optimized divisive hierarchical clustering."""
        customers_df = customers_df.copy().reset_index(drop=True)
        
        # Validate input
        if 'Latitude' not in customers_df.columns or 'Longitude' not in customers_df.columns:
            raise ValueError("DataFrame must contain 'Latitude' and 'Longitude' columns")
        
        coords_array = customers_df[['Latitude', 'Longitude']].values
        n_customers = len(customers_df)
        
        if n_customers == 0:
            customers_df['cluster'] = []
            return customers_df
        
        if n_customers == 1:
            customers_df['cluster'] = 1
            return customers_df
        
        # Priority queue approach for better clustering
        clusters_to_process = [(n_customers, np.arange(n_customers))]   # (size, indices)
        final_clusters = []
        
        iteration_count = 0
        max_iterations = n_customers * 2 # Safety break to prevent infinite loops
        
        while clusters_to_process and iteration_count < max_iterations:
            # Process largest cluster first to tackle the biggest problems
            clusters_to_process.sort(key=lambda x: x[0], reverse=True)
            current_size, current_cluster_indices = clusters_to_process.pop(0)
            iteration_count += 1
            
            if self.should_split_cluster(current_cluster_indices, coords_array):
                subclusters = self.geographic_split(current_cluster_indices, coords_array)
                
                for subcluster_indices in subclusters:
                    if len(subcluster_indices) > 0:
                        clusters_to_process.append((len(subcluster_indices), subcluster_indices))
            else:
                final_clusters.append(current_cluster_indices)
        
        # Handle any clusters remaining in `clusters_to_process` if max_iterations was hit
        final_clusters.extend([indices for _, indices in clusters_to_process])
        
        # Create result DataFrame
        result_df = customers_df.copy()
        result_df['cluster'] = -1 # Initialize with -1 for unassigned
        
        for cluster_id, cluster_indices in enumerate(final_clusters, 1):
            result_df.loc[cluster_indices, 'cluster'] = cluster_id
        
        return result_df





In [ ]:
df_all_cluster

In [ ]:
### My custom testing
print("--- Running Agglomerative Clustering ---")
agg_clusterer = AgglomerativeGeographicClustering(
    max_customers_per_cluster=20, # Aim for clusters of max 50 customers
    max_distance_km=5.0,        # Max diameter of 5 km
    linkage_method='ward',       # Common choice for compact clusters
    sub_cluster_if_oversized=True
)
clustered_agg_df = agg_clusterer.agglomerative_clustering(df_all_cluster.copy())
agg_stats = agg_clusterer.get_cluster_stats(clustered_agg_df, agg_clusterer.max_customers_per_cluster, agg_clusterer.max_distance_km)
print("\nAgglomerative Clustering Stats:")
for k, v in agg_stats['summary'].items():
    print(f"  {k}: {v}")
print("\nSample Agglomerative Cluster Details:")
for cluster_id, details in list(agg_stats.items())[:5]: # Show first 5 clusters
    if cluster_id != 'summary':
        print(f"  Cluster {cluster_id}: Size={details['size']}, Diameter={details['diameter_km']:.2f} km, Size OK={details['meets_size_constraint']}, Distance OK={details['meets_distance_constraint']}")


_ = evaluate_unsupervised_clustering(clustered_agg_df)

# # # Get detailed statistics
stats = divisive_clusterer.get_cluster_stats(clustered_agg_df)
print(f"Total clusters: {stats['summary']['total_clusters']}")
print(f"Constraint violations: Size={stats['summary']['size_violations']}, Distance={stats['summary']['distance_violations']}")
 

 
vis_and_save(df_routes = clustered_agg_df,
                 df_stockpoint = None,   
                 filename=None,
                 cluster_col = 'cluster')


In [ ]:
### My custom testing
print("\n--- Running Divisive Clustering (for comparison) ---")
div_clusterer = OptimizedDivisiveGeographicClustering(
    max_customers_per_cluster=50,
    max_distance_km=5.0,
    balance_clusters=True
)
clustered_div_df = div_clusterer.divisive_clustering(df_all_cluster.copy())
div_stats = div_clusterer.get_cluster_stats(clustered_div_df, div_clusterer.max_customers_per_cluster, div_clusterer.max_distance_km) # Added missing arguments
print("\nDivisive Clustering Stats:")
for k, v in div_stats['summary'].items():
    print(f"  {k}: {v}")
print("\nSample Divisive Cluster Details:")
for cluster_id, details in list(div_stats.items())[:5]: # Show first 5 clusters
    if cluster_id != 'summary':
        print(f"  Cluster {cluster_id}: Size={details['size']}, Diameter={details['diameter_km']:.2f} km, Size OK={details['meets_size_constraint']}, Distance OK={details['meets_distance_constraint']}")


_ = evaluate_unsupervised_clustering(clustered_div_df)

# # # Get detailed statistics
stats = divisive_clusterer.get_cluster_stats(clustered_div_df)
print(f"Total clusters: {stats['summary']['total_clusters']}")
print(f"Constraint violations: Size={stats['summary']['size_violations']}, Distance={stats['summary']['distance_violations']}")



vis_and_save(df_routes = clustered_div_df,
                df_stockpoint = None,   
                filename=None,
                cluster_col = 'cluster')


In [ ]:

# --- Example Usage ---
if __name__ == "__main__":
    # Generate some sample geographic data
    np.random.seed(42)
    num_customers = 500 # Testing with more customers for better demonstration
    
    # Simulate clusters
    center1 = [6.5, 3.3] # Lagos area
    center2 = [6.6, 3.4]
    center3 = [6.4, 3.2]

    customers_data = []
    # Cluster 1 (dense)
    for _ in range(200):
        customers_data.append({
            'CustomerID': f'C1_{_}',
            'Latitude': center1[0] + np.random.randn() * 0.01,
            'Longitude': center1[1] + np.random.randn() * 0.01
        })
    # Cluster 2 (dense)
    for _ in range(150):
        customers_data.append({
            'CustomerID': f'C2_{_}',
            'Latitude': center2[0] + np.random.randn() * 0.015,
            'Longitude': center2[1] + np.random.randn() * 0.015
        })
    # Cluster 3 (sparse, might get split or remain single if large enough)
    for _ in range(150):
        customers_data.append({
            'CustomerID': f'C3_{_}',
            'Latitude': center3[0] + np.random.randn() * 0.02,
            'Longitude': center3[1] + np.random.randn() * 0.02
        })

    customers_df = pd.DataFrame(customers_data)

    print("--- Running Agglomerative Clustering ---")
    agg_clusterer = AgglomerativeGeographicClustering(
        max_customers_per_cluster=50, # Aim for clusters of max 50 customers
        max_distance_km=5.0,        # Max diameter of 5 km
        linkage_method='ward',       # Common choice for compact clusters
        sub_cluster_if_oversized=True
    )
    clustered_agg_df = agg_clusterer.agglomerative_clustering(customers_df.copy())
    agg_stats = agg_clusterer.get_cluster_stats(clustered_agg_df, agg_clusterer.max_customers_per_cluster, agg_clusterer.max_distance_km)
    print("\nAgglomerative Clustering Stats:")
    for k, v in agg_stats['summary'].items():
        print(f"  {k}: {v}")
    print("\nSample Agglomerative Cluster Details:")
    for cluster_id, details in list(agg_stats.items())[:5]: # Show first 5 clusters
        if cluster_id != 'summary':
            print(f"  Cluster {cluster_id}: Size={details['size']}, Diameter={details['diameter_km']:.2f} km, Size OK={details['meets_size_constraint']}, Distance OK={details['meets_distance_constraint']}")

    print("\n--- Running Divisive Clustering (for comparison) ---")
    div_clusterer = OptimizedDivisiveGeographicClustering(
        max_customers_per_cluster=50,
        max_distance_km=5.0,
        balance_clusters=True
    )
    clustered_div_df = div_clusterer.divisive_clustering(customers_df.copy())
    div_stats = div_clusterer.get_cluster_stats(clustered_div_df, div_clusterer.max_customers_per_cluster, div_clusterer.max_distance_km) # Added missing arguments
    print("\nDivisive Clustering Stats:")
    for k, v in div_stats['summary'].items():
        print(f"  {k}: {v}")
    print("\nSample Divisive Cluster Details:")
    for cluster_id, details in list(div_stats.items())[:5]: # Show first 5 clusters
        if cluster_id != 'summary':
            print(f"  Cluster {cluster_id}: Size={details['size']}, Diameter={details['diameter_km']:.2f} km, Size OK={details['meets_size_constraint']}, Distance OK={details['meets_distance_constraint']}")

    # --- Plotting the clusters (optional, requires matplotlib/folium) ---
    # To visualize, you'd typically plot these clustered_agg_df or clustered_div_df
    # on a map using Folium, similar to our routing visualization.
    # For a quick visual check (requires matplotlib):
    try:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(15, 7))

        # Plot Agglomerative Clusters
        ax1 = plt.subplot(121)
        for cluster_id in clustered_agg_df['cluster'].unique():
            if cluster_id == -1: continue
            cluster_points = clustered_agg_df[clustered_agg_df['cluster'] == cluster_id]
            ax1.scatter(cluster_points['Longitude'], cluster_points['Latitude'], 
                        label=f'Agg C{cluster_id} (n={len(cluster_points)})', s=20, alpha=0.6)
        ax1.set_title('Agglomerative Clusters')
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.grid(True)

        # Plot Divisive Clusters
        ax2 = plt.subplot(122)
        for cluster_id in clustered_div_df['cluster'].unique():
            if cluster_id == -1: continue
            cluster_points = clustered_div_df[clustered_div_df['cluster'] == cluster_id]
            ax2.scatter(cluster_points['Longitude'], cluster_points['Latitude'], 
                        label=f'Div C{cluster_id} (n={len(cluster_points)})', s=20, alpha=0.6)
        ax2.set_title('Divisive Clusters')
        ax2.set_xlabel('Longitude')
        ax2.set_ylabel('Latitude')
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()

    except ImportError:
        print("\nMatplotlib not found. Skipping cluster visualization.")
    except Exception as e:
        print(f"\nError during plotting: {e}. Skipping cluster visualization.")







